# Setup

In [1]:
# Load modules
import pandas as pd
import os
import numpy as np
# from utils import affiliate_institutions, institution_name_transform, bin                                                                                                                  
from utils import *

# Set pandas dataframe options
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [2]:
# Define helper functions
def format_census_tract(tract_number, remove_decimal=False):
    if not tract_number or tract_number == '0000NAN':
        return tract_number
    
    # Convert to float
    tract_number = float(tract_number)
    
    # Format
    res = '%07.2F'% tract_number
    
    # Remove decimal
    if remove_decimal:
        return res.replace('.', '')
    else:
        return res



def sum_eth(row):
    return row['Asian'] + row['Black or African American'] +  row['Hispanic or Latino'] +  row['Other'] + row['White not Hispanic or Latino']


In [3]:
# Create folder for Excel exports
excel_exports_folder = "excel-exports"

if os.path.exists(excel_exports_folder) is False:
    os.mkdir(excel_exports_folder)
    print(f"Folder `{excel_exports_folder}` successfully created")

# **Unpickle dfs from pickle files**

## Census

In [4]:
census_df = pd.read_pickle(os.path.abspath(os.path.join('data-exports','census.pkl')))
census_df.head(5)

,tract,county,state,Total,Hispanic or Latino,Not Hispanic or Latino,Population of one race,White alone,Black or African American alone,American Indian and Alaska Native alone,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Population of two or more races,Population of two races,White; Black or African American,White; American Indian and Alaska Native,White; Asian,White; Native Hawaiian and Other Pacific Islander,White; Some Other Race,Black or African American; American Indian and Alaska Native,Black or African American; Asian,Black or African American; Native Hawaiian and Other Pacific Islander,Black or African American; Some Other Race,American Indian and Alaska Native; Asian,American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander,American Indian and Alaska Native; Some Other Race,Asian; Native Hawaiian and Other Pacific Islander,Asian; Some Other Race,Native Hawaiian and Other Pacific Islander; Some Other Race,Population of three races,White; Black or African American; American Indian and Alaska Native,White; Black or African American; Asian,White; Black or African American; Native Hawaiian and Other Pacific Islander,White; Black or African American; Some Other Race,White; American Indian and Alaska Native; Asian,White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander,White; American Indian and Alaska Native; Some Other Race,White; Asian; Native Hawaiian and Other Pacific Islander,White; Asian; Some Other Race,White; Native Hawaiian and Other Pacific Islander; Some Other Race,Black or African American; American Indian and Alaska Native; Asian,Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander,Black or African American; American Indian and Alaska Native; Some Other Race,Black or African American; Asian; Native Hawaiian and Other Pacific Islander,Black or African American; Asian; Some Other Race,Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race,American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,American Indian and Alaska Native; Asian; Some Other Race,American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Population of four races,White; Black or African American; American Indian and Alaska Native; Asian,White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander,White; Black or African American; American Indian and Alaska Native; Some Other Race,White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander,White; Black or African American; Asian; Some Other Race,White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race,White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,White; American Indian and Alaska Native; Asian; Some Other Race,White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,Black or African American; American Indian and Alaska Native; Asian; Some Other Race,Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Population of five races,White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,White; Black or African 

## FFIEC data

In [5]:
# Raw dataset
ffiec_data = pd.read_pickle(os.path.abspath(os.path.join('data-exports','ffiec_data.pkl')))

# FFIEC data preprocessing
ffiec_data['Tract Income Indicator'] = \
ffiec_data['Income indicator, which identifies low, moderate, middle, and upper income areas']\
.replace({0.:"Unknown Income", 1.:"Low or Moderate Income", 2.:"Low or Moderate Income",\
          3.:"Middle Income", 4.:"Upper Income"}) 

ffiec_data.head(5)

HMDA/CRA collection year MSA/MD Code  State         County Census tract  \
72753                     2022       19124  TEXAS  Collin County      0301.01   
72754                     2022       19124  TEXAS  Collin County      0301.02   
72755                     2022       19124  TEXAS  Collin County      0302.01   
72756                     2022       19124  TEXAS  Collin County      0302.02   
72757                     2022       19124  TEXAS  Collin County      0302.04   

      Principal city flag Small county flag  \
72753  not principal city      tract record   
72754  not principal city      tract record   
72755  not principal city      tract record   
72756  not principal city      tract record   
72757  not principal city      tract record   

                        Split tract flag  \
72753  tract number occurs within one MA   
72754  tract number occurs within one MA   
72755  tract number occurs within one MA   
72756  tract number occurs within one MA   
72757  tract number occurs within one MA   

                                   Demographic data flag Urban/rural flag  \
72753  total persons/population and median family inc...              nan   
72754  total persons/population and median family inc...              nan   
72755  total persons/population and median family inc...              nan   
72756  total persons/population and median family inc...              nan   
72757  total persons/population and median family inc...              nan   

       MSA/MD median family income  MSA/MD median household income  \
72753                      88315.0                         74251.0   
72754                      88315.0                         74251.0   
72755                      88315.0                         74251.0   
72756                      88315.0                         74251.0   
72757                      88315.0                         74251.0   

       Tract median family income as a percentage of the MSA/MD median family income. 2 decimal places, truncated.  \
72753                                              89.77                                                             
72754                                              76.70                                                             
72755                                             100.49                                                             
72756                                             163.98                                                             
72757                                             112.05                                                             

       FFIEC Estimated MSA/MD median family income  \
72753                                      97400.0   
72754                                      97400.0   
72755                                      97400.0   
72756                                      97400.0   
72757                                      97400.0   

       Income indicator, which identifies low, moderate, middle, and upper income areas  \
72753                                                3.0                                  
72754                                                2.0                                  
72755                                                3.0                                  
72756                                                4.0                                  
72757                                                3.0                                  

      CRA poverty criteria CRA unemployment criteria CRA distressed criteria  \
72753                   No                        No                      No   
72754                   No                        No                      No   
72755                   No                        No                      No   
72756                   No                        No                      No   
72757                   No                        No                      No   

      CRA remote rural (l

In [6]:
# Create a lookup table for FFIEC tract-level income indicators
ffiec_tract_income_lookup = ffiec_data[['State', 'County', 'Census tract', 'Tract Income Indicator']].drop_duplicates().reset_index()

# Insert census tract ID with decimal removed
ffiec_tract_income_lookup['Census_Tract_No_Decimal'] = ffiec_tract_income_lookup['Census tract'].apply(lambda x: x.replace(".", ""))

ffiec_tract_income_lookup.head(10)

,index,State,County,Census tract,Tract Income Indicator,Census_Tract_No_Decimal
0,72753,TEXAS,Collin County,0301.01,Middle Income,030101
1,72754,TEXAS,Collin County,0301.02,Low or Moderate Income,030102
2,72755,TEXAS,Collin County,0302.01,Middle Income,030201
3,72756,TEXAS,Collin County,0302.02,Upper Income,030202
4,72757,TEXAS,Collin County,0302.04,Middle Income,030204
5,72758,TEXAS,Collin County,0302.05,Middle Income,030205
6,72759,TEXAS,Collin County,0302.06,Upper Income,030206
7,72760,TEXAS,Collin County,0302.07,Middle Income,030207
8,72761,TEXAS,Collin County,0303.01,Upper Income,030301
9,72762,TEXAS,Collin County,0303.02,Upper Income,030302


In [7]:
# Export FFIEC tract-level income indicators to Excel
out_file_path = os.path.join(excel_exports_folder, "ffiec_tract_income_lookup.xlsx")
export_to_excel(ffiec_tract_income_lookup, out_file_path, sheet_name="ffiec_tract_income_lookup", startcol=0, startrow=0)

Exporting dataframe to excel-exports/ffiec_tract_income_lookup.xlsx


## HMDA - Loan Application Record (LAR)

In [8]:
# Raw data
hmda_dict_lar_df = pd.read_pickle(os.path.abspath(os.path.join('data-exports','hmda_dict_lar_df.pkl')))

# List of categorical variables
category_lst = ['derived_loan_product_type', 'loan_type','loan_purpose','lien_status',\
                'open_end_line_of_credit','business_or_commercial_purpose',\
                'debt_to_income_ratio','applicant_credit_score_type','applicant_sex',\
                'applicant_age']

# Combine low frequency categories
bin_category(hmda_dict_lar_df, category_lst, 200)

# binning certain values to further preprocess hmda data 
dti_replace_map = {dti_val:'36%-<50%' for dti_val in hmda_dict_lar_df['debt_to_income_ratio']  if str(dti_val) in str(set(range(36,50)))}
hmda_dict_lar_df['debt_to_income_ratio'] = hmda_dict_lar_df['debt_to_income_ratio'].replace(dti_replace_map).replace({np.nan:'Unknown'})
hmda_dict_lar_df['applicant_age'] = hmda_dict_lar_df['applicant_age'].replace({'8888':'Not applicable'})

hmda_dict_lar_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4
1,2022,RVDPPPGHCGZ40J4VQ731,19124,TEXAS,Collin County,0313.14,NaN,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,Loan originated,Ginnie Mae,Preapproval not requested,Federal Housing Administration insured (FHA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,445000,92.74,6.75,0.847,Not a high-cost mortgage,16155.19,NaN,4290.4,3276.4,NaN,360.0,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,475000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,202.0,36%-<50%,Not applicable,Experian Fair Isaac,Not Hispanic or Latino,NaN,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Black or African American,White,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Male,"Information not provided by applicant in mail,...",Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,45-54,35-44,No,No,Not submitted directly to your institution,Initially payable to your institution,Technology Open to Approved Lenders (TOTAL) Sc...,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14,NaN
27,2022,RVDPPPGHCGZ40J4VQ731,23104,TEXAS,Tarrant County,1115.63,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Joint,Application withdrawn by applicant,Not applicable,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,235000,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,NaN,NaN,36

### HMDA Panel Data

In [9]:
hmda_dict_panel_df = pd.read_pickle(os.path.abspath(os.path.join('data-exports','hmda_dict_panel_df.pkl')))
hmda_dict_panel_df.head(5)

,activity_year,lei,tax_id,agency_code,id_2017,respondent_rssd,respondent_name,respondent_state,respondent_city,assets,other_lender_code,parent_rssd,parent_name,topholder_rssd,topholder_name
0,2022,5493009YXZEN8325UC34,46-3986756,Department of Housing and Urban Development,NaN,4877394.0,"HOME MORTGAGE ALLIANCE, LLC",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,B4TYDEB6GKMZO031MB27,94-1687665,Consumer Financial Protection Bureau,480228,480228.0,"Bank of America, National Association",NC,Charlotte,2.519525e+09,NaN,1379552.0,BANA HOLDING CORPORATION,1073757.0,BANK OF AMERICA CORPORATION
2,2022,5493000YNV8IX4VD3X12,62-0997810,Department of Housing and Urban Development,1936928,1936928.0,"VANDERBILT MORTGAGE AND FINANCE, INC.",TN,KNOXVILLE,NaN,Independent mortgage banking subsidiary,NaN,NaN,NaN,NaN
3,2022,549300VZVN841I2ILS84,57-1175755,Department of Housing and Urban Development,4183602,4183602.0,"CROSSCOUNTRY MORTGAGE, INC.",OH,AKRON,NaN,Independent mortgage banking subsidiary,NaN,NaN,NaN,NaN
4,2022,549300CB67L6KPJLHE19,59-1515932,Department of Housing and Urban Development,4320902,4320902.0,"TRIAD FINANCIAL SERVICES, INC.",FL,JACKSONVILLE,NaN,Independent mortgage banking subsidiary,NaN,NaN,NaN,NaN


In [10]:
hmda_dict_panel_df['agency_code'].value_counts()

agency_code
National Credit Union Administration           1465
Federal Deposit Insurance Corporation          1184
Department of Housing and Urban Development     925
Office of the Comptroller of the Currency       440
Federal Reserve System                          312
Consumer Financial Protection Bureau            141
Name: count, dtype: int64

# CRA Data
- Transmittal Sheet
- Disclosure D1-1 Data

Joined on respondent ID and agency code

In [11]:
# Transmittal sheet
cra_2022_transmittal_sheet = pd.read_pickle(os.path.abspath(os.path.join('data-exports','CRA2022_Transmittal.pkl')))
cra_2022_transmittal_sheet.head(5)

,Respondent ID,Agency Code,Activity Year,Institution name,Institution address,Institution city,Institution state,Institution zip code,Tax ID,ID_RSSD,Institution assets
0,0000000001,OCC,2022,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
1,0000000008,OCC,2022,"JPMORGAN CHASE BANK, NA",1111 POLARIS PARKWAY,COLUMBUS,OH,43240,13-4994650,852218,3306982000
2,0000000024,OCC,2022,US BANK NA,777 E WISCONSIN AVE,MILWAUKEE,WI,53202,31-0841368,504713,564154604
3,0000000047,OCC,2022,"FIRST FINANCIAL BANK, NA","1401 S THIRD ST, PO BOX 2122",TERRE HAUTE,IN,47802,35-0704860,693345,5045573
4,0000000209,OCC,2022,FIRST NATIONAL BANK OF OMAHA,1620 DODGE STREET,OMAHA,NE,68197,47-0259043,527954,26812859


In [12]:
# CRA Discl D11
cra2022_Discl_D11_df_raw = pd.read_pickle(os.path.abspath(os.path.join('data-exports','cra2022_Discl_D11.pkl')))
cra2022_Discl_D11_df_raw.head(5)

,Table ID,Respondent ID,Agency Code,Activity Year,Loan Type,Action Taken Type,State,County,MSA/MD,Assessment Area Number,Partial County Indicator,Split County Indicator,Population Classification,Income Group Total,Report Level,"Number of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Number of Small Business Loans Originated with Loan Amount at Origination > 100,000 and < or = to $250,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $100,000 and < or = to $250,000","Number of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000",Number of Loans Originated to Small Businesses with Gross Annual Revenues < $1 million,Total Loan Amount of Loans Originated to Small Businesses with Gross Annual Revenues < or = to $1 million,Number of Small Business Loans Originated Reported as Affiliate Loans,Total Loan Amount of Small Business Loans Originated Reported as Affiliate Loans
17073,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",< 10% of Median Family Income(MFI),not a total,0,0,0,0,0,0,0,0,0,0
17074,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",10% to 20% of MFI,not a total,0,0,0,0,0,0,0,0,0,0
17075,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",20% to 30% of MFI,not a total,0,0,0,0,0,0,0,0,0,0
17076,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",30% to 40% of MFI,not a total,0,0,0,0,0,0,0,0,0,0
17077,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,not a total,1,91000,1,102000,0,0,0,0,2,193000


In [13]:
# Join transmittal sheet to 2022 cra d11 file 
cra2022_Discl_D11_df = pd.merge(cra2022_Discl_D11_df_raw,
         cra_2022_transmittal_sheet,
         left_on = ['Respondent ID','Agency Code', 'Activity Year'],
         right_on = ['Respondent ID','Agency Code', 'Activity Year'],
         how = 'left')


cra2022_Discl_D11_df.head(5)

,Table ID,Respondent ID,Agency Code,Activity Year,Loan Type,Action Taken Type,State,County,MSA/MD,Assessment Area Number,Partial County Indicator,Split County Indicator,Population Classification,Income Group Total,Report Level,"Number of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Number of Small Business Loans Originated with Loan Amount at Origination > 100,000 and < or = to $250,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $100,000 and < or = to $250,000","Number of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000",Number of Loans Originated to Small Businesses with Gross Annual Revenues < $1 million,Total Loan Amount of Loans Originated to Small Businesses with Gross Annual Revenues < or = to $1 million,Number of Small Business Loans Originated Reported as Affiliate Loans,Total Loan Amount of Small Business Loans Originated Reported as Affiliate Loans,Institution name,Institution address,Institution city,Institution state,Institution zip code,Tax ID,ID_RSSD,Institution assets
0,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",< 10% of Median Family Income(MFI),not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
1,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",10% to 20% of MFI,not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
2,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",20% to 30% of MFI,not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
3,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",30% to 40% of MFI,not a total,0,0,0,0,0,0,0,0,0,0,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
4,D1-1,0000000001,OCC,2022,Small Business,Originations,TEXAS,Collin County,19124.0,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,not a total,1,91000,1,102000,0,0,0,0,2,193000,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000


In [14]:
# subset of cra data for dashboard
cra_d11_cols = [i for i in cra2022_Discl_D11_df_raw.columns.to_list() if 'Number of ' in i or 'Total Loan ' in i]

cra_2022_D11_subset = cra2022_Discl_D11_df[[
    'Respondent ID',
    'Agency Code',
    'Institution name',
    'Institution address',
    'Institution city',
    'Institution state',
    'Institution zip code',
    'Institution assets',
    'County',
    'Population Classification',
    'Income Group Total']\
+ cra_d11_cols].reset_index()

# reset index to list counting from 0 to len of range of original index col
cra_2022_D11_subset['index'] = list(range(0,len(cra_2022_D11_subset['index'])))

# set index on new index column
cra_2022_D11_subset = cra_2022_D11_subset.set_index('index')

# get indices of full d11 dataframe as set
all_d11_indices = set(cra_2022_D11_subset.index.to_list())

# get indices of subset d11 df containing rows that have totals as entries
d11_totals_indices = set(cra_2022_D11_subset[cra_2022_D11_subset['Population Classification'] == 'total'].index.to_list())

# find indices from full d11 df that do not have totals entires
d11_indices_no_totals = list(all_d11_indices - d11_totals_indices)
cra_2022_D11_subset = cra_2022_D11_subset.iloc[d11_indices_no_totals,:]

cra_2022_D11_subset.head(5)


,Respondent ID,Agency Code,Institution name,Institution address,Institution city,Institution state,Institution zip code,Institution assets,County,Population Classification,Income Group Total,"Number of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination < or = to $100,000","Number of Small Business Loans Originated with Loan Amount at Origination > 100,000 and < or = to $250,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $100,000 and < or = to $250,000","Number of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000","Total Loan Amount of Small Business Loans Originated with Loan Amount at Origination > $250,000 and < or = to $1,000,000",Number of Loans Originated to Small Businesses with Gross Annual Revenues < $1 million,Total Loan Amount of Loans Originated to Small Businesses with Gross Annual Revenues < or = to $1 million,Number of Small Business Loans Originated Reported as Affiliate Loans,Total Loan Amount of Small Business Loans Originated Reported as Affiliate Loans
index,,,,,,,,,,,,,,,,,,,,,
0,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",< 10% of Median Family Income(MFI),0,0,0,0,0,0,0,0,0,0
1,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",10% to 20% of MFI,0,0,0,0,0,0,0,0,0,0
2,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",20% to 30% of MFI,0,0,0,0,0,0,0,0,0,0
3,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",30% to 40% of MFI,0,0,0,0,0,0,0,0,0,0
4,0000000001,OCC,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,1779504000,Collin County,"counties with >500,000 in population",40% to 50% of MFI,1,91000,1,102000,0,0,0,0,2,193000


## CRA Disclosure D-6

In [15]:
# Raw data
cra2022_Discl_D6_df_raw = pd.read_pickle(os.path.abspath(os.path.join('data-exports','cra2022_Discl_D6.pkl')))

# Merge with transmittal sheet
cra2022_Discl_D6_df = pd.merge(cra2022_Discl_D6_df_raw,
         cra_2022_transmittal_sheet,
         left_on = ['Respondent ID', 'Agency Code', 'Activity Year'],
         right_on = ['Respondent ID', 'Agency Code', 'Activity Year'],
         how = 'left')

cra2022_Discl_D6_df.head(5)

,Table ID,Respondent ID,Agency Code,Activity Year,State,County,MSA/MD,Census Tract,Assessment Area Number,Partial County Indicator,Split County Indicator,Population Classification,Income Group,Loan Indicator,Filler,Institution name,Institution address,Institution city,Institution state,Institution zip code,Tax ID,ID_RSSD,Institution assets
0,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0317.20,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
1,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0317.23,427.0,No,No,"counties with >500,000 in population",40% to 50% of MFI,No,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
2,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0304.09,427.0,No,No,"counties with >500,000 in population",50% to 60% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
3,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0320.12,427.0,No,No,"counties with >500,000 in population",50% to 60% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000
4,D6-0,0000000001,OCC,2022,TEXAS,Collin County,19124.0,0305.40,427.0,No,No,"counties with >500,000 in population",60% to 70% of MFI,Yes,NaN,"WELLS FARGO BANK, N.A.",420 MONTGOMERY STREET,SAN FRANCISCO,CA,94104,94-1347393,451965,1779504000


In [16]:
# Income group lookup
sel_cols = ['County', 'Census Tract', 'Income Group']
cra_income_group_lookup = cra2022_Discl_D6_df[sel_cols].drop_duplicates()
cra_income_group_lookup = cra_income_group_lookup.sort_values(sel_cols).reset_index(drop=True)
# Rename cra_income_group_lookup 'Income Group' column to 'CRA Tract Income Group'
cra_income_group_lookup.rename(columns={'Income Group': 'CRA Tract Income Group'}, inplace=True)
cra_income_group_lookup.head(5)

,County,Census Tract,CRA Tract Income Group
0,Collin County,0301.01,80% to 90% of MFI
1,Collin County,0301.02,70% to 80% of MFI
2,Collin County,0302.01,100% to 110% of MFI
3,Collin County,0302.02,> 120% of MFI
4,Collin County,0302.04,110% to 120% of MFI


## FDIC Institutions & Locations

In [17]:
# FDIC institutions
fdic_institutions_df = pd.read_pickle(os.path.abspath(os.path.join('data-exports','fdic_institutions_df.pkl')))
fdic_institutions_df['FDIC Certificate #'] = fdic_institutions_df['FDIC Certificate #'].apply(float)
fdic_institutions_df.head(5)

,FDIC Certificate #,index,State Name,OTS Docket Number,Institution Status,Street Address,Total assets,Institution Class,Change Code 1,Change Code 2,Change Code 3,Change Code 4,Change Code 5,Change Code 6,Change Code 7,Change Code 8,Change Code 9,Change Code 10,Change Code 11,Change Code 12,Change Code 13,Change Code 14,Change Code 15,OCC Charter Number,Chartering Agency,Conservatorship,City,Numeric Code,Consolidated Metropolitan Statistical Division Number,Consolidated Metropolitan Statistical Area,County,Last update,Denovo Institution,Total deposits,Last Structure Change Effective Date,End date,Equity capital,Established Date,FDIC Geographic Region,FDIC Supervisory Region,Federal Reserve District,Federal Reserve ID Number,Federal Reserve ID Number 2,Federal charter flag,FDIC Field Office,Insured offices of foreign banks,Inactive,Primary Insurance Agency,Secondary Insurance Fund,Date of Deposit Insurance,Credit Card Institutions,Bank Insurance Fund,Insured commercial banks,Deposit Insurance Fund member,FDIC Insured,SAIF Insured,Insured Savings Institution,Metropolitan Statistical Area Number,Metropolitan Statistical Area (MSA),Institution name,New certificate number,Oakar Institutions,OTS District,Office of Thrift Supervision Region,Last Structure Change Process Date,Quarterly Banking Profile Commercial Bank Region,Primary Regulator,Report Date,Report Date,State Charter,Return on assets (ROA),Quarterly return on assets,Return on Equity (ROE),Quarterly return on equity,Run Date,Sasser Institutions,Law Sasser Flag,State Alpha code,State and county number,State Number,Zip Code,Supervisory Region Number,Office of the Comptroller,FDIC's unique number,Ultimate Cert,CFPB Effective Date,CFPB End Date,CFPB Flag,Secondary Regulator,Web Site URL 01,Web Site URL 02,Web Site URL 03,Web Site URL 04,Web Site URL 05,Web Site URL 06,Web Site URL 07,Web Site URL 08,Web Site URL 09,Web Site URL 10,Trade Name 01,Trade Name 02,Trade Name 03,Trade Name 04,Trade Name 05,Trade Name 06,Primary Internet Web Address,Office,Directly owned by another bank (CERT),Directly owned by another bank (CERT),City of High Holder,Deposits held in domestic offices,FFIEC Call Report 31 Filer,Bank Holding Company Type,Agricultural lending institution indicator,Ownership Type,Bank Holding Company (Regulatory Top Holder),Net income,Net income - quarterly,Number of Domestic Offices,Number of Foreign Offices,Number of US Offices,RSSDID - High Regulatory Holder,Regulatory holding company state location,Subchapter S Corporations,Pretax return on assets,Quarterly Pretax return on assets,Trust Powers,Asset Concentration Hierarchy,Specialization Group,Institutions with reportable fiduciary related service,Combined Statistical Area Name,Combined Statistical Area Number (Branch),CSA Area Flag,Core Based Statistical Area Name,Core Based Statistical Areas,Metropolitan Division Name,Metropolitan Division Number,Metropolitan Division Flag,Micropolitan Division Flag,Metropolitan Divisions Name,Metropolitan Divisions Number,Metropolitan Divisions Flag,Community Bank
0,17772.0,0,Ohio,13876,Institution closed or not insured by FDIC,7001 Center Street,208001.0,"Commercial bank, state charter, Fed non-member...",223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,STATE,NaN,Mentor,NaN,28,"Cleveland-Akron, OH",Lake,01/14/2004,0.0,193300.0,12/13/2003,12/13/2003,14234.0,1957-06-13,9,Chicago,Chicago,Cleveland,592822,0,Columbus,0,1,BIF,NaN,06/13/1957,0.0,1,1,No,1,0,0,1680,"Cleveland-Lorain-Elyria, OH",Great Lakes Bank,5982,0,3.0,Central,01/14/2004,3,Federal Deposit Insurance Corporation,09/30/2003,06/30/2023,yes,0.774869,1.01,10.87,14.30,09/29/2023,0,NaN,OH,39085,39,44060,Chicago,3,11726,6560,9999-12-31,9999-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,MENTOR,193300.0,no,NaN,0.0,0.0,GLB BCORP INC,1112.0,500.0,13.0,0.0,0.0,2571755.0,OH,0.0,1.179723,1.53,0.0,4.0,Commercial Lending Specialization,0.0,"Cleveland-Akron-Canton, OH",184.0,NaN

In [18]:
# FDIC locations
fdic_locations_df = pd.read_pickle(os.path.abspath(os.path.join('data-exports','fdic_locations_df.pkl')))
fdic_locations_df.head(5)

,index,Branch Address,Institution Class,Core Based Statistical Area Name (Branch),Metropolitan Divisions Name (Branch),Metropolitan Divisions Flag (Branch),Metropolitan Divisions Number (Branch),Metropolitan Division Number (Branch),Metropolitan Division Flag (Branch),Metropolitan Division Name (Branch),Micropolitan Division Flag (Branch),Core Based Statistical Areas (Branch),Institution FDIC Certificate #,Branch City,Branch County,Combined Statistical Area Name (Branch),Combined Statistical Area Flag (Branch),Combined Statistical Area Number (Branch),Branch Established Date,FDIC UNINUM of the Owner Institution,Main Office,Institution Name,Office Name,Branch Number,Run Date,Service Type Code,Branch State Abbreviation,State and County Number,Branch State,Unique Identification Number for a Branch Office,Branch Zip Code,County_from_Zipcode,idx_branch_number,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,Branch identifier,Branch street address,Match result,Match type
0,56,12524 N Beach St,"Commercial bank, national (federal) charter an...","Dallas-Fort Worth-Arlington, TX","Fort Worth-Arlington-Grapevine, TX",Yes,23104.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,3510,Fort Worth,Tarrant County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2013-10-07,2238,0,"Bank of America, National Association",TIMBERLAND CROSSING BRANCH,9744,09/29/2023,NaN,TX,48439,Texas,544684,76244,Tarrant County,(56) 9744,TEXAS,1139.42,3019,Tarrant County,32.954916250849465,-97.28116256017745,{'latitute': '32.954916250849465'},(56) 9744,"12524 N Beach St, Fort Worth, TX, 76244",Match,Exact
1,57,6015 Weldon Howell Pkwy,"Commercial bank, national (federal) charter an...","Dallas-Fort Worth-Arlington, TX","Dallas-Plano-Irving, TX",Yes,19124.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,628,Dallas,Dallas County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2013-10-04,417,0,"JPMorgan Chase Bank, National Association",PRESTON CENTER MOTOR BANK BRANCH,7827,09/29/2023,NaN,TX,48113,Texas,544713,75225,Dallas County,(57) 7827,TEXAS,0073.01,2027,Dallas County,32.861180685718466,-96.80820366420835,{'latitute': '32.861180685718466'},(57) 7827,"6015 Weldon Howell Pkwy, Dallas, TX, 75225",Match,Exact
2,76,6925 Preston Rd,"Commercial bank, state charter and Fed member,...","Dallas-Fort Worth-Arlington, TX","Dallas-Plano-Irving, TX",Yes,19124.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,12614,Dallas,Dallas County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2014-01-10,8010,0,Origin Bank,"BTH BANK, A DIVISION OF BANKTEXAS NA BRANCH",73,09/29/2023,NaN,TX,48113,Texas,557401,75205,Dallas County,(76) 73,TEXAS,0195.02,1018,Dallas County,32.84936695631433,-96.80432344619965,{'latitute': '32.84936695631433'},(76) 73,"6925 Preston Rd, Dallas, TX, 75205",Match,Exact
3,171,100 Crescent Ct,"Commercial bank, national (federal) charter an...","Dallas-Fort Worth-Arlington, TX","Dallas-Plano-Irving, TX",Yes,19124.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,91325,Dallas,Dallas County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2015-11-01,18316,0,CIBC National Trust Company,CIBC NATIONAL TRUST COMPANY DALLAS BRANCH,15,09/29/2023,NaN,TX,48113,Texas,580574,75201,Dallas County,(171) 15,TEXAS,0018.02,1021,Dallas County,32.79446647285579,-96.80331932817637,{'latitute': '32.79446647285579'},(171) 15,"100 Crescent Ct, Dallas, TX, 75201",Match,Exact
4,208,5825 Edwards Ranch Rd,"Commercial bank, state charter and Fed member,...","Dallas-Fort Worth-Arlington, TX","Fort Worth-Arlington-Grapevine, TX",Yes,23104.0,19100,Yes,"Dallas-Fort Worth-Arlington, TX",No,19100,57414,Fort Worth,Tarrant County,"Dallas-Fort Worth, TX-OK",Yes,206.0,2017-07-24,363318,0,Worthington Bank,EDWARDS RANCH BRANCH,3,09/29/2023,NaN,TX,48439,Texas,590336,76109,Tarrant County,(208) 3,TEXAS,1054.05,1023,Tarrant County,32.70783983911434,-97.41039592845334,{'latitute': '32.70783983911434'},(208) 3,"5825 Edwards Ranch Rd, Fort Worth, TX, 76109",Match,Exact


## SBA Data

In [19]:
# Raw data
sba_data = pd.read_pickle(os.path.abspath(os.path.join('data-exports','sba_data.pkl')))

# subset for 'Paid In Full', 'Charged Off', and 'The status of loans that have been disbursed but have not been cancelled,
# paid in full, or charged off are exempt from disclosure under FOIA Exemption 4'

sba_data = sba_data[(sba_data['LoanStatus'] == 'Paid In Full') |\
(sba_data['LoanStatus'] == 'Charged Off') |\
(sba_data['LoanStatus'] == 'The status of loans that have been disbursed but have not been cancelled, paid in full, \
or charged off are exempt from disclosure under FOIA Exemption 4')]

# recoding CongressionalDistrict as a string
sba_data['CongressionalDistrict'] = sba_data['CongressionalDistrict'].apply(lambda dist: f"District {int(dist):02d}")

sba_category_lst = ['DeliveryMethod','NaicsDescription','CongressionalDistrict','BusinessType','BusinessAge']

# binning all unique entires within category column of sba data under .01
bin_category(sba_data, sba_category_lst, 0.01, True,'proportion')

sba_data = sba_data.rename(columns = {'BankName':'institution_name'})

sba_data.head(5)

,index,AsOfDate,Program,Borrower name,Borrower street address_x,Borrower city,Borrower state,Borrower zip code,institution_name,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,Total loan amount,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram description,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SOLDSECMRTIND,County_from_Zipcode,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,Borrower identifier,Borrower street address_y,Match result,Match type,county_final
14,103266,20230930,7A,(103266) Forester Haynie PLLC,400 SAINT PAUL ST Suite 700,DALLAS,TEXAS,75201,Texas Security Bank,58739.0,NaN,1212 Turtle Creek Blvd,DALLAS,TX,75207,200000.0,100000.0,2022-01-04,2022,01/28/2022,SBA Express,FA$TRK (Small Loan Express),5.25,12,541110.0,Other,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 30,CORPORATION,Existing or more than 2 years old,Paid In Full,05/31/2022,NaN,0,Revolver,3,NaN,Dallas County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dallas County
18,103339,20230930,7A,(103339) Aspire Technical Solutions LL,1475 HERITAGE PKWY SUITE 201,MANSFIELD,TEXAS,76063,Celtic Bank Corporation,57056.0,NaN,"268 S State St, Ste 300",SALT LAKE CITY,UT,84111,150000.0,127500.0,2022-01-05,2022,01/14/2022,Preferred Lender Program,Guaranty,6.00,120,541512.0,Other,NaN,NaN,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 06,CORPORATION,Existing or more than 2 years old,The status of loans that have been disbursed b...,NaN,NaN,0,Term,2,Sold on the secondary market,Tarrant County,TEXAS,1113.09,2003,Tarrant County,32.55325719311065,-97.12222046413416,{'latitute': '32.55325719311065'},(103339) Aspire Technical Solutions LL,"1475 HERITAGE PKWY SUITE 201, MANSFIELD, TEXAS...",Match,Exact,Tarrant County
19,103341,20230930,7A,(103341) Kmandy Investments L.L.C.,6509 San Judas Street,McKinney,TEXAS,75070,Celtic Bank Corporation,57056.0,NaN,"268 S State St, Ste 300",SALT LAKE CITY,UT,84111,586700.0,440025.0,2022-01-05,2022,01/11/2022,Preferred Lender Program,Guaranty,6.00,120,454210.0,Other,S2056,WaterStation,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 03,CORPORATION,"Startup, Loan Funds will Open Business",The status of loans that have been disbursed b...,NaN,NaN,0,Term,1,Sold on the secondary market,Collin County,TEXAS,0305.40,2006,Collin County,33.15427189246418,-96.70470560393005,{'latitute': '33.15427189246418'},(103341) Kmandy Investments L.L.C.,"6509 San Judas Street, McKinney, TEXAS, 75070",Match,Exact,Collin County
22,103362,20230930,7A,(103362) SAHER AUTO REPAIR INC,255 WEST CAMP WISDOM ROAD,DUNCANVILLE,TEXAS,75116,IncredibleBank,19772.0,NaN,327 N 17th Ave,WAUSAU,WI,54401,290000.0,217500.0,2022-01-05,2022,01/28/2022,Preferred Lender Program,Guaranty,5.75,120,811111.0,General Automotive Repair,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 30,CORPORATION,Change of Ownership,The status of loans that have been disbursed b...,NaN,NaN,0,Term,15,Sold on the secondary market,Dallas County,TEXAS,0165.35,2008,Dallas County,32.66196359730873,-96.91000812554029,{'latitute': '32.66196359730873'},(103362) SAHER AUTO REPAIR INC,"255 WEST CAMP WISDOM ROAD, DUNCANVILLE, TEXAS,...",Match,Exact,Dallas County
26,103429,20230930,7A,(103429) Broadmoore SRG PLLC,"519 N Cedar Ridge Dr., Ste 1",DUNCANVILLE,TEXAS,75116,First National Bank Texas,3285.0,NaN,901 E Central Texas Expy,Killeen,TX,76541,751500.0,563625.0,2022-01-05,2022,01/20/2022,Preferred Lender Program,Guaranty,4.35,120,621210.0,Offices of Dentists,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 30,CORPORATION,New Business or 2 years or less,The status of loans that have been disbursed b...,NaN,NaN,0,Term,4,NaN,Dallas County,TEXAS,0165

# **Joins**

## hmda_panel to hmda_lar

In [20]:
hmda_dict_panel_df_sub = hmda_dict_panel_df[['lei','respondent_name']]

hmda_lar_panel_df = pd.merge(\
    hmda_dict_lar_df, hmda_dict_panel_df_sub,\
    left_on = ['lei'], right_on = ['lei'], how = 'left')\
.rename(columns = {'respondent_name':'institution_name'})

hmda_lar_panel_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,institution_name
0,2022,RVDPPPGHCGZ40J4VQ731,19124,TEXAS,Collin County,0313.14,NaN,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,Loan originated,Ginnie Mae,Preapproval not requested,Federal Housing Administration insured (FHA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,445000,92.74,6.75,0.847,Not a high-cost mortgage,16155.19,NaN,4290.4,3276.4,NaN,360.0,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,475000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,202.0,36%-<50%,Not applicable,Experian Fair Isaac,Not Hispanic or Latino,NaN,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Black or African American,White,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Male,"Information not provided by applicant in mail,...",Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,45-54,35-44,No,No,Not submitted directly to your institution,Initially payable to your institution,Technology Open to Approved Lenders (TOTAL) Sc...,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14,NaN,"PENNYMAC LOAN SERVICES, LLC"
1,2022,RVDPPPGHCGZ40J4VQ731,23104,TEXAS,Tarrant County,1115.63,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Joint,Application withdrawn by applicant,Not applicable,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,235000,NaN

In [21]:
# below shows so we do not have a matching bank name for every lei 
print('lar',hmda_dict_lar_df['lei'].nunique(),'\nmerged panel and lar',\
      hmda_lar_panel_df['institution_name'].nunique())

lar 1123 
merged panel and lar 1113


In [22]:
# BANK NAME INTO ORIGINAL LAR DF 

hmda_dict_lar_df = pd.merge(\
    hmda_dict_lar_df, hmda_dict_panel_df_sub,\
    left_on = ['lei'], right_on = ['lei'], how = 'left')\
.rename(columns = {'respondent_name':'institution_name'})

hmda_dict_lar_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,institution_name
0,2022,RVDPPPGHCGZ40J4VQ731,19124,TEXAS,Collin County,0313.14,NaN,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,Loan originated,Ginnie Mae,Preapproval not requested,Federal Housing Administration insured (FHA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,445000,92.74,6.75,0.847,Not a high-cost mortgage,16155.19,NaN,4290.4,3276.4,NaN,360.0,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,475000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,202.0,36%-<50%,Not applicable,Experian Fair Isaac,Not Hispanic or Latino,NaN,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Black or African American,White,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Male,"Information not provided by applicant in mail,...",Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,45-54,35-44,No,No,Not submitted directly to your institution,Initially payable to your institution,Technology Open to Approved Lenders (TOTAL) Sc...,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14,NaN,"PENNYMAC LOAN SERVICES, LLC"
1,2022,RVDPPPGHCGZ40J4VQ731,23104,TEXAS,Tarrant County,1115.63,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Joint,Application withdrawn by applicant,Not applicable,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,235000,NaN

# **Add Race_Ethnicity field to HDMA data** 

if the race is in (Asian, asian indian, other asian, vietnamese, filipino, Korean, Japanese) then asian 

if hmda_dict_lar_df['applicant_race_1'] == "White" and "Hispanic or Latino" in hmda_dict_lar_df['applicant_ethnicity_1'] then hmda_dict_lar_df['applicant_race_1'] +' '+ hmda_dict_lar_df['applicant_ethnicity_1']

if race == "Black or African American" then "Black or African American"

else "Other"

In [23]:
# old: applicant_race_1, applicant_ethnicity_1

race_field = 'derived_race'
eth_field = 'derived_ethnicity'

hmda_dict_lar_df['Race_Ethnicity'] = np.where(\
    (hmda_dict_lar_df[race_field] == "Black or African American"),"Black or African American",\
    np.where(\
        ((hmda_dict_lar_df[eth_field] == "Hispanic or Latino")|\
        (hmda_dict_lar_df[eth_field] == "Other Hispanic or Latino")),"Hispanic or Latino",\
        np.where(\
            (hmda_dict_lar_df['applicant_race_1'] == "White"), "White not Hispanic or Latino",\
            np.where(\
                ((hmda_dict_lar_df[race_field] == "Asian")|\
                 (hmda_dict_lar_df[race_field] == "Asian Indian")|\
                 (hmda_dict_lar_df[race_field] == "Other Asian")|\
                 (hmda_dict_lar_df[race_field] == "Vietnamese")|\
                 (hmda_dict_lar_df[race_field] == "Filipino")|\
                 (hmda_dict_lar_df[race_field] == "Korean")|\
                 (hmda_dict_lar_df[race_field] == "Japanese")), "Asian", "Other"
            )
        )
         
    )
        )

hmda_dict_lar_df.head(5)

,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,derived_race,derived_sex,action_taken,purchaser_type,preapproval,loan_type,loan_purpose,lien_status,reverse_mortgage,open_end_line_of_credit,business_or_commercial_purpose,loan_amount,combined_loan_to_value_ratio,interest_rate,rate_spread,hoepa_status,total_loan_costs,total_points_and_fees,origination_charges,discount_points,lender_credits,loan_term,prepayment_penalty_term,intro_rate_period,negative_amortization,interest_only_payment,balloon_payment,other_nonamortizing_features,property_value,construction_method,occupancy_type,manufactured_home_secured_property_type,manufactured_home_land_property_interest,total_units,multifamily_affordable_units,income,debt_to_income_ratio,applicant_credit_score_type,co_applicant_credit_score_type,applicant_ethnicity_1,applicant_ethnicity_2,applicant_ethnicity_3,applicant_ethnicity_4,applicant_ethnicity_5,co_applicant_ethnicity_1,co_applicant_ethnicity_2,co_applicant_ethnicity_3,co_applicant_ethnicity_4,co_applicant_ethnicity_5,applicant_ethnicity_observed,co_applicant_ethnicity_observed,applicant_race_1,applicant_race_2,applicant_race_3,applicant_race_4,applicant_race_5,co_applicant_race_1,co_applicant_race_2,co_applicant_race_3,co_applicant_race_4,co_applicant_race_5,applicant_race_observed,co_applicant_race_observed,applicant_sex,co_applicant_sex,applicant_sex_observed,co_applicant_sex_observed,applicant_age,co_applicant_age,applicant_age_above_62,co_applicant_age_above_62,submission_of_application,initially_payable_to_institution,aus_1,aus_2,aus_3,aus_4,aus_5,denial_reason_1,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units,aus-4,institution_name,Race_Ethnicity
0,2022,RVDPPPGHCGZ40J4VQ731,19124,TEXAS,Collin County,0313.14,NaN,FHA:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Black or African American,Male,Loan originated,Ginnie Mae,Preapproval not requested,Federal Housing Administration insured (FHA),Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a business or commercial pur...,445000,92.74,6.75,0.847,Not a high-cost mortgage,16155.19,NaN,4290.4,3276.4,NaN,360.0,NaN,NaN,No negative amortization,No interest-only payments,No balloon payment,No other non-fully amortizing features,475000.0,Site-built,Principal residence,Not applicable,Not applicable,1,NaN,202.0,36%-<50%,Not applicable,Experian Fair Isaac,Not Hispanic or Latino,NaN,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Black or African American,White,NaN,NaN,NaN,"Information not provided by applicant in mail,...",NaN,NaN,NaN,NaN,Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,Male,"Information not provided by applicant in mail,...",Not collected on the basis of visual observati...,Not collected on the basis of visual observati...,45-54,35-44,No,No,Not submitted directly to your institution,Initially payable to your institution,Technology Open to Approved Lenders (TOTAL) Sc...,NaN,NaN,NaN,NaN,Not applicable,NaN,NaN,NaN,6001,39.96,97400,150.0,1255,1431,14,NaN,"PENNYMAC LOAN SERVICES, LLC",Black or African American
1,2022,RVDPPPGHCGZ40J4VQ731,23104,TEXAS,Tarrant County,1115.63,NaN,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,Asian,Joint,Application withdrawn by applicant,Not applicable,Preapproval not requested,Conventional (not insured or guaranteed by FHA...,Home purchase,Secured by a first lien,Not a reverse mortgage,Not an open-end line of credit,Not primarily for a

In [24]:
hmda_dict_lar_df['Race_Ethnicity'].value_counts()

Race_Ethnicity
White not Hispanic or Latino    85985
Other                           71269
Hispanic or Latino              35981
Asian                           30192
Black or African American       25871
Name: count, dtype: int64

# **Summary Tables** 

## Branch-level details

In [25]:
fdic_branch_details = fdic_locations_df.copy()[['Institution Name','Office Name','Branch Address', 'Branch City',\
                   'Branch State', 'Branch Zip Code', '_tract', 'Branch County']]

fdic_branch_details.rename(columns={'Institution Name': 'institution_name'}, inplace=True) 
fdic_branch_details = clean_inst_names(fdic_branch_details)

fdic_branch_details.head(5)

,institution_name,Office Name,Branch Address,Branch City,Branch State,Branch Zip Code,_tract,Branch County
0,BANK OF AMERICA NA,TIMBERLAND CROSSING BRANCH,12524 N Beach St,Fort Worth,Texas,76244,1139.42,Tarrant County
1,JPMORGAN CHASE BANK NA,PRESTON CENTER MOTOR BANK BRANCH,6015 Weldon Howell Pkwy,Dallas,Texas,75225,0073.01,Dallas County
2,ORIGIN BANK,"BTH BANK, A DIVISION OF BANKTEXAS NA BRANCH",6925 Preston Rd,Dallas,Texas,75205,0195.02,Dallas County
3,CIBC NATIONAL TRUST COMPANY,CIBC NATIONAL TRUST COMPANY DALLAS BRANCH,100 Crescent Ct,Dallas,Texas,75201,0018.02,Dallas County
4,WORTHINGTON BANK,EDWARDS RANCH BRANCH,5825 Edwards Ranch Rd,Fort Worth,Texas,76109,1054.05,Tarrant County


In [26]:
# Use the format_census_tract function to format the census tract column
fdic_branch_details['tract_fmt'] = fdic_branch_details['_tract'].apply(format_census_tract)
fdic_branch_details['tract_fmt_no_decimal'] = fdic_branch_details['_tract'].apply(format_census_tract, remove_decimal=True)
fdic_branch_details.head()


,institution_name,Office Name,Branch Address,Branch City,Branch State,Branch Zip Code,_tract,Branch County,tract_fmt,tract_fmt_no_decimal
0,BANK OF AMERICA NA,TIMBERLAND CROSSING BRANCH,12524 N Beach St,Fort Worth,Texas,76244,1139.42,Tarrant County,1139.42,113942
1,JPMORGAN CHASE BANK NA,PRESTON CENTER MOTOR BANK BRANCH,6015 Weldon Howell Pkwy,Dallas,Texas,75225,0073.01,Dallas County,0073.01,007301
2,ORIGIN BANK,"BTH BANK, A DIVISION OF BANKTEXAS NA BRANCH",6925 Preston Rd,Dallas,Texas,75205,0195.02,Dallas County,0195.02,019502
3,CIBC NATIONAL TRUST COMPANY,CIBC NATIONAL TRUST COMPANY DALLAS BRANCH,100 Crescent Ct,Dallas,Texas,75201,0018.02,Dallas County,0018.02,001802
4,WORTHINGTON BANK,EDWARDS RANCH BRANCH,5825 Edwards Ranch Rd,Fort Worth,Texas,76109,1054.05,Tarrant County,1054.05,105405


In [131]:
# Export FDIC Branch Details to Excel
out_file_path = os.path.join(excel_exports_folder, "fdic_branch_details.xlsx")
export_to_excel(fdic_branch_details, out_file_path, sheet_name="fdic_branch_details", startcol=0, startrow=0)

# Alternative if it throws IllegalCharacterError
# !pip install xlsxwriter
# fdic_branch_details.to_excel(out_file_path, engine='xlsxwriter')

Exporting dataframe to excel-exports/fdic_branch_details.xlsx


## **Census tract-level** summaries

In [28]:
# Get race counts by census tract
# Doesn't include Hispanic or Latino ethnicity consideration

# ffiec_data columns: 

# Asian - Total population Asian
# Asian - Total population White and Asian
# Asian - Total population American Indian or Alaska native
# Asian - Total population White and Native Hawaiian or other Pacific Islander
# Asian - Total population Native Hawaiian or other Pacific Islander
# Asian - Total population White and American Indian or Alaska Native

# Black - Total population Black/African American
# Black - Total population White and Black
# Black - Total population Black and American Indian or Alaska Native

# Other - instead of using `Total population some other race`, compute as `Total population` - `Total Population Asian` - `Total Population Black` - `Total Population White`

# White - Total population White
# White - Total population White and some other race

# ffiec_data['Total Population Asian'] = ffiec_data['Total population Asian'] + ffiec_data['Total population White and Asian'] + \
#     ffiec_data['Total population American Indian or Alaska native'] + \
#     ffiec_data['Total population White and Native Hawaiian or other Pacific Islander'] + \
#     ffiec_data['Total population Native Hawaiian or other Pacific Islander']

# ffiec_data['Total Population Black'] = ffiec_data['Total population Black/African American'] + \
#     ffiec_data['Total population White and Black'] + ffiec_data['Total population Black and American Indian or Alaska Native']

# ffiec_data['Total Population White'] = ffiec_data['Total population White'] + ffiec_data['Total population White and some other race']

# ffiec_data['Total population'] = ffiec_data['Total population by race']

# ffiec_data['Total Population Other'] = ffiec_data['Total population'] - ffiec_data['Total Population Asian'] - \
#     ffiec_data['Total Population Black'] - ffiec_data['Total Population White']

In [29]:
# ffiec_intermed_df = ffiec_data.groupby(['County','Census tract'], as_index = False)\
# [['Total persons', 'Total families','Total households','Tract Income Indicator', 
#   'Total Population Asian', 'Total Population Black', 'Total Population Other', 'Total Population White', 'Total population']].sum()

# ffiec_intermed_df.head(5)

In [30]:
# # Compute percentage of each racial group, multiply by 100 and round to 2 decimal places
# ffiec_intermed_df['Percent Asian'] = (ffiec_intermed_df['Total Population Asian'] / ffiec_intermed_df['Total population']) * 100
# ffiec_intermed_df['Percent Black or African American'] = (ffiec_intermed_df['Total Population Black'] / ffiec_intermed_df['Total population']) * 100
# ffiec_intermed_df['Percent Hispanic or Latino'] = (ffiec_intermed_df['Total Population Other'] / ffiec_intermed_df['Total population']) * 100
# ffiec_intermed_df['Percent White not Hispanic or Latino'] = (ffiec_intermed_df['Total Population White'] / ffiec_intermed_df['Total population']) * 100

# # Round each percentage to 2 decimal places
# ffiec_intermed_df['Percent Asian'] = ffiec_intermed_df['Percent Asian'].round(2)
# ffiec_intermed_df['Percent Black or African American'] = ffiec_intermed_df['Percent Black or African American'].round(2)
# ffiec_intermed_df['Percent Hispanic or Latino'] = ffiec_intermed_df['Percent Hispanic or Latino'].round(2)
# ffiec_intermed_df['Percent White not Hispanic or Latino'] = ffiec_intermed_df['Percent White not Hispanic or Latino'].round(2)

# # Compute the sum of the racial percentages without using sum_eth
# ffiec_intermed_df['Sum of Racial Percentages'] = ffiec_intermed_df['Percent Asian'] + ffiec_intermed_df['Percent Black or African American'] + \
#     ffiec_intermed_df['Percent Hispanic or Latino'] + ffiec_intermed_df['Percent White not Hispanic or Latino']

# ffiec_intermed_df.head(5)


In [31]:
# hmda originated
hmda_loan_count = hmda_dict_lar_df[hmda_dict_lar_df['action_taken'] == 'Loan originated']\
.groupby(['county_code','census_tract'],as_index = False)\
[['lei']].count().rename(columns = {'lei':'number_of_hmda_loans'})

hmda_loan_count.head(5)

,county_code,census_tract,number_of_hmda_loans
0,Collin County,0301.01,113
1,Collin County,0301.02,140
2,Collin County,0302.01,585
3,Collin County,0302.02,960
4,Collin County,0302.04,579


In [32]:
# HMDA loan counts by race / ethnicity within each county, census tract, and institution

hmda_race_sex_count_df = hmda_dict_lar_df[hmda_dict_lar_df['action_taken'] == 'Loan originated']\
.groupby(['county_code', 'census_tract', 'institution_name', 'derived_sex'])\
[['Race_Ethnicity']]\
.value_counts(normalize = False)\
.unstack()\
.reset_index()

# Standardize institution names
hmda_race_sex_count_df = clean_inst_names(hmda_race_sex_count_df)

# Census tract formatted without decimal
hmda_race_sex_count_df.insert(2, 'Census_Tract_no_decimal', hmda_race_sex_count_df['census_tract'].str.replace('.',''))

# Replace NA counts with zeroes
values = {'Asian': 0.0, 
          'Black or African American': 0.0,
          'Hispanic or Latino': 0.0, 
          'Other': 0.0, 
          'White not Hispanic or Latino': 0.0}
hmda_race_sex_count_df = hmda_race_sex_count_df.fillna(values)

# Sum across race/ethnicity
hmda_race_sex_count_df['Total'] = hmda_race_sex_count_df.apply(sum_eth, axis=1)

hmda_race_sex_count_df.head(5)

Race_Ethnicity,county_code,census_tract,Census_Tract_no_decimal,institution_name,derived_sex,Asian,Black or African American,Hispanic or Latino,Other,White not Hispanic or Latino,Total
0,Collin County,0301.01,030101,TEXASBANK,Joint,0.0,0.0,0.0,0.0,1.0,1.0
1,Collin County,0301.01,030101,21ST MORTGAGE CORPORATION,Joint,0.0,0.0,0.0,0.0,2.0,2.0
2,Collin County,0301.01,030101,21ST MORTGAGE CORPORATION,Male,0.0,0.0,0.0,0.0,1.0,1.0
3,Collin County,0301.01,030101,AMERICAN FINANCIAL NETWORK INC,Joint,0.0,0.0,0.0,0.0,1.0,1.0
4,Collin County,0301.01,030101,ANGEL OAK HOME LOANS LLC,Male,0.0,0.0,0.0,0.0,1.0,1.0


In [33]:
# Export HMDA race counts to Excel
out_file_path = os.path.join(excel_exports_folder, "hmda_race_sex_count_by_inst.xlsx")
export_to_excel(hmda_race_sex_count_df, out_file_path, sheet_name="hmda_race_sex_count_df", startcol=0, startrow=0)

Exporting dataframe to excel-exports/hmda_race_sex_count_by_inst.xlsx


# FFIEC data summaries

In [34]:
base_ffiec_columns = ['County', 'Census tract', 'Total persons', 'Total families', 'Total households', 'Tract Income Indicator']

ffiec_race_cnt_cols = ['Total Population Asian', 'Total Population Black', 'Total Population Other', 'Total Population White', 
'Total population']

ffiec_race_pct_cols = ['Percent Asian', 'Percent Black or African American', 'Percent Hispanic or Latino', 'Percent White not Hispanic or Latino']



In [35]:
# Show the last 20 columns of the FFIEC data
ffiec_data.iloc[:, -20:].head(5)

,"Total owner-occupied units $50,000-$59,999","Total owner-occupied units $60,000-$69,999","Total owner-occupied units $70,000-$79,999","Total owner-occupied units $80,000-$89,999","Total owner-occupied units $90,000-$99,999","Total owner-occupied units $100,000-$124,999","Total owner-occupied units $125,000-$149,999","Total owner-occupied units $150,000-$174,999","Total owner-occupied units $175,000-$199,999","Total owner-occupied units $200,000-$249,999","Total owner-occupied units $250,000-$299,999","Total owner-occupied units $300,000-$399,999","Total owner-occupied units $400,000-$499,999","Total owner-occupied units $500,000-$749,999","Total owner-occupied units $750,000-$999,999","Total owner-occupied units $1,000,000 to $1,499,999","Total owner-occupied units $1,500,000 to $1,999,999","Total owner-occupied units $2,000,000 or more",Median value in dollars for all owner-occupied housing units,Tract Income Indicator
72753,2.0,10.0,59.0,0.0,9.0,51.0,87.0,41.0,149.0,58.0,16.0,64.0,54.0,30.0,1.0,0.0,0.0,0.0,183600.0,Middle Income
72754,19.0,48.0,42.0,28.0,3.0,88.0,28.0,43.0,60.0,28.0,39.0,155.0,46.0,100.0,13.0,13.0,0.0,0.0,198500.0,Low or Moderate Income
72755,5.0,0.0,15.0,25.0,19.0,24.0,20.0,27.0,24.0,107.0,129.0,165.0,142.0,112.0,66.0,5.0,7.0,0.0,324200.0,Middle Income
72756,0.0,7.0,0.0,18.0,17.0,14.0,0.0,24.0,15.0,37.0,120.0,310.0,197.0,139.0,54.0,24.0,5.0,2.0,366900.0,Upper Income
72757,0.0,0.0,0.0,0.0,15.0,6.0,60.0,200.0,215.0,646.0,183.0,142.0,0.0,0.0,0.0,0.0,0.0,0.0,217200.0,Middle Income


## FFIEC Version 1: Using race fields only (no Hispanic ethnicity consideration)

In [36]:
# Compute the sum of the racial populations
ffiec_data['Total Population Asian'] = ffiec_data['Total population Asian'] + ffiec_data['Total population White and Asian'] + \
    ffiec_data['Total population American Indian or Alaska native'] + \
    ffiec_data['Total population White and Native Hawaiian or other Pacific Islander'] + \
    ffiec_data['Total population Native Hawaiian or other Pacific Islander']

ffiec_data['Total Population Black'] = ffiec_data['Total population Black/African American'] + \
    ffiec_data['Total population White and Black'] + ffiec_data['Total population Black and American Indian or Alaska Native']

ffiec_data['Total Population White'] = ffiec_data['Total population White'] + ffiec_data['Total population White and some other race']

ffiec_data['Total population'] = ffiec_data['Total population by race']

ffiec_data['Total Population Other'] = ffiec_data['Total population'] - ffiec_data['Total Population Asian'] - \
    ffiec_data['Total Population Black'] - ffiec_data['Total Population White']

In [37]:
# Selected columns from ffiec_data for joins
ffiec_subset = ffiec_data[base_ffiec_columns + ffiec_race_cnt_cols].copy()

# Compute percentages for each racial group
ffiec_subset['Percent Asian'] = (ffiec_subset['Total Population Asian'] / ffiec_subset['Total population']) * 100
ffiec_subset['Percent Black or African American'] = (ffiec_subset['Total Population Black'] / ffiec_subset['Total population']) * 100
ffiec_subset['Percent White'] = (ffiec_subset['Total Population White'] / ffiec_subset['Total population']) * 100
ffiec_subset['Percent Other'] = (ffiec_subset['Total Population Other'] / ffiec_subset['Total population']) * 100

# Round each percentage to 2 decimal places
ffiec_subset['Percent Asian'] = ffiec_subset['Percent Asian'].round(2)
ffiec_subset['Percent Black or African American'] = ffiec_subset['Percent Black or African American'].round(2)
ffiec_subset['Percent White'] = ffiec_subset['Percent White'].round(2)
ffiec_subset['Percent Other'] = ffiec_subset['Percent Other'].round(2)

# Join to cra_income_group_lookup
ffiec_subset = pd.merge(ffiec_subset, cra_income_group_lookup, how='left', 
left_on=['County', 'Census tract'],
right_on=['County', 'Census Tract'])


ffiec_subset.head(5)

,County,Census tract,Total persons,Total families,Total households,Tract Income Indicator,Total Population Asian,Total Population Black,Total Population Other,Total Population White,Total population,Percent Asian,Percent Black or African American,Percent White,Percent Other,Census Tract,CRA Tract Income Group
0,Collin County,0301.01,3595.0,596.0,750.0,Middle Income,101.0,59.0,467.0,2968.0,3595.0,2.81,1.64,82.56,12.99,0301.01,80% to 90% of MFI
1,Collin County,0301.02,4282.0,831.0,1011.0,Low or Moderate Income,117.0,61.0,556.0,3548.0,4282.0,2.73,1.42,82.86,12.98,0301.02,70% to 80% of MFI
2,Collin County,0302.01,6175.0,1024.0,1154.0,Middle Income,343.0,668.0,517.0,4647.0,6175.0,5.55,10.82,75.26,8.37,0302.01,100% to 110% of MFI
3,Collin County,0302.02,6345.0,983.0,1186.0,Upper Income,416.0,585.0,411.0,4933.0,6345.0,6.56,9.22,77.75,6.48,0302.02,> 120% of MFI
4,Collin County,0302.04,7554.0,2019.0,2354.0,Middle Income,300.0,1044.0,822.0,5388.0,7554.0,3.97,13.82,71.33,10.88,0302.04,110% to 120% of MFI


In [38]:
# Export FFIEC subset to Excel
out_file_path = os.path.join(excel_exports_folder, "ffiec_cra_pop_and_income_by_tract.xlsx")
export_to_excel(ffiec_subset, out_file_path, sheet_name="ffiec_cra_pop_and_income_by_tr", startcol=0, startrow=0)

Exporting dataframe to excel-exports/ffiec_cra_pop_and_income_by_tract.xlsx


## FFIEC Version 2: Using Hispanic and non-Hispanic by Race fields [Preferred]

In [39]:
ffiec_data_summary = ffiec_data.copy()
# hispanic
ffiec_data_summary['Total Population Hispanic']\
= ffiec_data_summary['Total population Hispanic or Latino by race']

# white
ffiec_data_summary['Total Population White']\
= ffiec_data_summary['Total population non-Hispanic White']

# black
ffiec_data_summary['Total Population Black']\
= ffiec_data_summary\
['Total population non-Hispanic White and Black'] +\
ffiec_data_summary\
['Total population non-Hispanic Black/African American'] +\
ffiec_data_summary\
['Total population non-Hispanic Black and American Indian or Alaska native']

# asian
ffiec_data_summary['Total Population Asian']\
= ffiec_data_summary\
['Total population non-Hispanic Asian']+\
    ffiec_data_summary\
['Total population non-Hispanic native Hawaiian or other Pacific Islander']+\
    ffiec_data_summary\
['Total population non-Hispanic White and Asian']+\
    ffiec_data_summary\
['Total population non-Hispanic White and Native Hawaiian or other Pacific Islander']

# total population
ffiec_data_summary['Total Population'] = ffiec_data_summary['Total population Hispanic and Non-Hispanic']

# other
ffiec_data_summary['Total Population Other'] = ffiec_data_summary['Total Population'] - \
    ffiec_data_summary['Total Population Hispanic'] - ffiec_data_summary['Total Population White'] - \
        ffiec_data_summary['Total Population Black'] - ffiec_data_summary['Total Population Asian']


ffiec_data_summary['Calc Total'] = ffiec_data_summary['Total Population Hispanic'] + ffiec_data_summary['Total Population White'] + \
    ffiec_data_summary['Total Population Black'] + ffiec_data_summary['Total Population Asian'] + ffiec_data_summary['Total Population Other']

base_ffiec_columns = ['County', 'Census tract', 'Total persons', 'Total families', 'Total households', 'Tract Income Indicator']
ffiec_race_cnt_cols = ['Total Population Asian', 'Total Population Black', 'Total Population Hispanic', 'Total Population Other', 
                       'Total Population White', 'Total Population']
ffiec_data_summary = ffiec_data_summary[base_ffiec_columns + ffiec_race_cnt_cols + ['Calc Total']]
    
ffiec_data_summary.head(5)

,County,Census tract,Total persons,Total families,Total households,Tract Income Indicator,Total Population Asian,Total Population Black,Total Population Hispanic,Total Population Other,Total Population White,Total Population,Calc Total
72753,Collin County,0301.01,3595.0,596.0,750.0,Middle Income,53.0,56.0,782.0,194.0,2510.0,3595.0,3595.0
72754,Collin County,0301.02,4282.0,831.0,1011.0,Low or Moderate Income,49.0,57.0,832.0,205.0,3139.0,4282.0,4282.0
72755,Collin County,0302.01,6175.0,1024.0,1154.0,Middle Income,255.0,648.0,905.0,293.0,4074.0,6175.0,6175.0
72756,Collin County,0302.02,6345.0,983.0,1186.0,Upper Income,366.0,570.0,850.0,242.0,4317.0,6345.0,6345.0
72757,Collin County,0302.04,7554.0,2019.0,2354.0,Middle Income,206.0,1013.0,1705.0,298.0,4332.0,7554.0,7554.0


In [40]:
# Sanity check... do the totals match?
ffiec_data_summary[['Total Population','Calc Total']].head(5)

,Total Population,Calc Total
72753,3595.0,3595.0
72754,4282.0,4282.0
72755,6175.0,6175.0
72756,6345.0,6345.0
72757,7554.0,7554.0


In [41]:
# subset for values of interest
ffiec_data_summary = ffiec_data_summary[[
    'County',
    'Census tract', 'Tract Income Indicator',
    'Total persons', 'Total families', 'Total households',
    'Total Population Hispanic',
    'Total Population White',
    'Total Population Black',
    'Total Population Asian',
    'Total Population Other',
    'Total Population'
    ]]

In [42]:
ffiec_data_summary['Census_Tract_no_decimal'] = ffiec_data_summary['Census tract'].str.replace('.','')

# Create a new column in ffiec_data_summary called 'data_source' and set it to 'FFIEC'
# Use a method that avoids SettingWithCopyWarning
#ffiec_data_summary = ffiec_data_summary.assign(data_source='FFIEC')
ffiec_data_summary.head(5)

,County,Census tract,Tract Income Indicator,Total persons,Total families,Total households,Total Population Hispanic,Total Population White,Total Population Black,Total Population Asian,Total Population Other,Total Population,Census_Tract_no_decimal
72753,Collin County,0301.01,Middle Income,3595.0,596.0,750.0,782.0,2510.0,56.0,53.0,194.0,3595.0,030101
72754,Collin County,0301.02,Low or Moderate Income,4282.0,831.0,1011.0,832.0,3139.0,57.0,49.0,205.0,4282.0,030102
72755,Collin County,0302.01,Middle Income,6175.0,1024.0,1154.0,905.0,4074.0,648.0,255.0,293.0,6175.0,030201
72756,Collin County,0302.02,Upper Income,6345.0,983.0,1186.0,850.0,4317.0,570.0,366.0,242.0,6345.0,030202
72757,Collin County,0302.04,Middle Income,7554.0,2019.0,2354.0,1705.0,4332.0,1013.0,206.0,298.0,7554.0,030204


In [43]:
# Merge ffiec_data_summary with cra_income_group_lookup
ffiec_data_summary = pd.merge(ffiec_data_summary, cra_income_group_lookup, 
                              left_on=['County', 'Census tract'], 
                              right_on=['County', 'Census Tract'], how='left') 

In [44]:
# ffiec_race_pct_cols = ['Percent Asian', 'Percent Black', 'Percent Hispanic', 'Percent White', 'Percent Other']

# Compute Percentage of each racial group, multiply by 100 and round to 2 decimal places
ffiec_data_summary['Percent Hispanic'] = (ffiec_data_summary['Total Population Hispanic'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent White'] = (ffiec_data_summary['Total Population White'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent Black'] = (ffiec_data_summary['Total Population Black'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent Asian'] = (ffiec_data_summary['Total Population Asian'] / ffiec_data_summary['Total Population']) * 100
ffiec_data_summary['Percent Other'] = (ffiec_data_summary['Total Population Other'] / ffiec_data_summary['Total Population']) * 100

# Round each percentage to 2 decimal places
ffiec_data_summary['Percent Hispanic'] = ffiec_data_summary['Percent Hispanic'].round(2)
ffiec_data_summary['Percent White'] = ffiec_data_summary['Percent White'].round(2)
ffiec_data_summary['Percent Black'] = ffiec_data_summary['Percent Black'].round(2)
ffiec_data_summary['Percent Asian'] = ffiec_data_summary['Percent Asian'].round(2)
ffiec_data_summary['Percent Other'] = ffiec_data_summary['Percent Other'].round(2)

ffiec_data_summary.head(5)


,County,Census tract,Tract Income Indicator,Total persons,Total families,Total households,Total Population Hispanic,Total Population White,Total Population Black,Total Population Asian,Total Population Other,Total Population,Census_Tract_no_decimal,Census Tract,CRA Tract Income Group,Percent Hispanic,Percent White,Percent Black,Percent Asian,Percent Other
0,Collin County,0301.01,Middle Income,3595.0,596.0,750.0,782.0,2510.0,56.0,53.0,194.0,3595.0,030101,0301.01,80% to 90% of MFI,21.75,69.82,1.56,1.47,5.40
1,Collin County,0301.02,Low or Moderate Income,4282.0,831.0,1011.0,832.0,3139.0,57.0,49.0,205.0,4282.0,030102,0301.02,70% to 80% of MFI,19.43,73.31,1.33,1.14,4.79
2,Collin County,0302.01,Middle Income,6175.0,1024.0,1154.0,905.0,4074.0,648.0,255.0,293.0,6175.0,030201,0302.01,100% to 110% of MFI,14.66,65.98,10.49,4.13,4.74
3,Collin County,0302.02,Upper Income,6345.0,983.0,1186.0,850.0,4317.0,570.0,366.0,242.0,6345.0,030202,0302.02,> 120% of MFI,13.40,68.04,8.98,5.77,3.81
4,Collin County,0302.04,Middle Income,7554.0,2019.0,2354.0,1705.0,4332.0,1013.0,206.0,298.0,7554.0,030204,0302.04,110% to 120% of MFI,22.57,57.35,13.41,2.73,3.94


In [45]:
# Export FFIEC data summary to Excel
out_file_path = os.path.join(excel_exports_folder, "ffiec_data_summary_wide.xlsx")
export_to_excel(ffiec_data_summary, out_file_path, sheet_name="ffiec_data_summary", startcol=0, startrow=0)

Exporting dataframe to excel-exports/ffiec_data_summary_wide.xlsx


# Intermediate dataframes for merge

In [46]:
# Use tract income lookup as the final dataframe for FFIEC data
ffiec_intermed_df = ffiec_tract_income_lookup.copy()

ffiec_intermed_df.head(5)

,index,State,County,Census tract,Tract Income Indicator,Census_Tract_No_Decimal
0,72753,TEXAS,Collin County,0301.01,Middle Income,030101
1,72754,TEXAS,Collin County,0301.02,Low or Moderate Income,030102
2,72755,TEXAS,Collin County,0302.01,Middle Income,030201
3,72756,TEXAS,Collin County,0302.02,Upper Income,030202
4,72757,TEXAS,Collin County,0302.04,Middle Income,030204


## Interm calc 1: HMDA loan originations

In [47]:
# hmda originated
hmda_loans_orig = hmda_dict_lar_df[hmda_dict_lar_df['action_taken'] == 'Loan originated'].copy()
hmda_loans_orig = clean_inst_names(hmda_loans_orig)


hmda_intermed_df = hmda_loans_orig\
.groupby(['county_code','census_tract', 'institution_name'],as_index = False)\
[['lei']].count().rename(columns = {'lei':'number_of_hmda_loans'})

hmda_intermed_df.head(3)

,county_code,census_tract,institution_name,number_of_hmda_loans
0,Collin County,0301.01,21ST MORTGAGE CORPORATION,3
1,Collin County,0301.01,ALLIANCE BANK,1
2,Collin County,0301.01,AMERICAN BANK OF COMMERCE,2


In [143]:
# Write a function to Group ahl_funding_loans by Race_Ethnicity and count the number of loans for a given institution name
def count_loans_by_race(institution_name):
    print(f"\nCount loans by Race/Ethnicity for {institution_name}\n")
    return hmda_loans_orig[hmda_loans_orig['institution_name'] == institution_name]\
    .groupby('Race_Ethnicity').size()

count_loans_by_race('AHL FUNDING')


Count loans by Race/Ethnicity for AHL FUNDING



Race_Ethnicity
Asian                           4
Black or African American       2
Hispanic or Latino              1
Other                           3
White not Hispanic or Latino    1
dtype: int64

In [144]:
count_loans_by_race('AAXY LLC')


Count loans by Race/Ethnicity for AAXY LLC



Race_Ethnicity
Asian                           17
Black or African American        1
Hispanic or Latino               1
White not Hispanic or Latino     5
dtype: int64

In [145]:
count_loans_by_race('ALL CREDIT CONSIDERED MORTGAGE INC')


Count loans by Race/Ethnicity for ALL CREDIT CONSIDERED MORTGAGE INC



Race_Ethnicity
Black or African American       3
Hispanic or Latino              9
Other                           1
White not Hispanic or Latino    6
dtype: int64

## Interm calc 2: SBA

TO DO: Run SBA data through Google Sheets Geocode to get census tracts?

In [133]:
sba_clean = clean_inst_names(sba_data)

sba_clean['Census_Tract_no_decimal'] = sba_clean['_tract'].apply(format_census_tract, remove_decimal=True)

sba_clean.head(5)

,index,AsOfDate,Program,Borrower name,Borrower street address_x,Borrower city,Borrower state,Borrower zip code,institution_name,BankFDICNumber,BankNCUANumber,BankStreet,BankCity,BankState,BankZip,Total loan amount,SBAGuaranteedApproval,ApprovalDate,ApprovalFiscalYear,FirstDisbursementDate,DeliveryMethod,Subprogram description,InitialInterestRate,TermInMonths,NaicsCode,NaicsDescription,FranchiseCode,FranchiseName,ProjectCounty,ProjectState,SBADistrictOffice,CongressionalDistrict,BusinessType,BusinessAge,LoanStatus,PaidInFullDate,ChargeOffDate,GrossChargeOffAmount,RevolverStatus,JobsSupported,SOLDSECMRTIND,County_from_Zipcode,_fips_state_name,_tract,_block,_fips_county_name,_latitude,_longitude,extensions,Borrower identifier,Borrower street address_y,Match result,Match type,county_final,Census_Tract_no_decimal
14,103266,20230930,7A,(103266) Forester Haynie PLLC,400 SAINT PAUL ST Suite 700,DALLAS,TEXAS,75201,TEXAS SECURITY BANK,58739.0,NaN,1212 Turtle Creek Blvd,DALLAS,TX,75207,200000.0,100000.0,2022-01-04,2022,01/28/2022,SBA Express,FA$TRK (Small Loan Express),5.25,12,541110.0,Other,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 30,CORPORATION,Existing or more than 2 years old,Paid In Full,05/31/2022,NaN,0,Revolver,3,NaN,Dallas County,NaN,0000NAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dallas County,0000NAN
18,103339,20230930,7A,(103339) Aspire Technical Solutions LL,1475 HERITAGE PKWY SUITE 201,MANSFIELD,TEXAS,76063,CELTIC BANK CORPORATION,57056.0,NaN,"268 S State St, Ste 300",SALT LAKE CITY,UT,84111,150000.0,127500.0,2022-01-05,2022,01/14/2022,Preferred Lender Program,Guaranty,6.00,120,541512.0,Other,NaN,NaN,TARRANT,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 06,CORPORATION,Existing or more than 2 years old,The status of loans that have been disbursed b...,NaN,NaN,0,Term,2,Sold on the secondary market,Tarrant County,TEXAS,1113.09,2003,Tarrant County,32.55325719311065,-97.12222046413416,{'latitute': '32.55325719311065'},(103339) Aspire Technical Solutions LL,"1475 HERITAGE PKWY SUITE 201, MANSFIELD, TEXAS...",Match,Exact,Tarrant County,111309
19,103341,20230930,7A,(103341) Kmandy Investments L.L.C.,6509 San Judas Street,McKinney,TEXAS,75070,CELTIC BANK CORPORATION,57056.0,NaN,"268 S State St, Ste 300",SALT LAKE CITY,UT,84111,586700.0,440025.0,2022-01-05,2022,01/11/2022,Preferred Lender Program,Guaranty,6.00,120,454210.0,Other,S2056,WaterStation,COLLIN,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 03,CORPORATION,"Startup, Loan Funds will Open Business",The status of loans that have been disbursed b...,NaN,NaN,0,Term,1,Sold on the secondary market,Collin County,TEXAS,0305.40,2006,Collin County,33.15427189246418,-96.70470560393005,{'latitute': '33.15427189246418'},(103341) Kmandy Investments L.L.C.,"6509 San Judas Street, McKinney, TEXAS, 75070",Match,Exact,Collin County,030540
22,103362,20230930,7A,(103362) SAHER AUTO REPAIR INC,255 WEST CAMP WISDOM ROAD,DUNCANVILLE,TEXAS,75116,INCREDIBLEBANK,19772.0,NaN,327 N 17th Ave,WAUSAU,WI,54401,290000.0,217500.0,2022-01-05,2022,01/28/2022,Preferred Lender Program,Guaranty,5.75,120,811111.0,General Automotive Repair,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 30,CORPORATION,Change of Ownership,The status of loans that have been disbursed b...,NaN,NaN,0,Term,15,Sold on the secondary market,Dallas County,TEXAS,0165.35,2008,Dallas County,32.66196359730873,-96.91000812554029,{'latitute': '32.66196359730873'},(103362) SAHER AUTO REPAIR INC,"255 WEST CAMP WISDOM ROAD, DUNCANVILLE, TEXAS,...",Match,Exact,Dallas County,016535
26,103429,20230930,7A,(103429) Broadmoore SRG PLLC,"519 N Cedar Ridge Dr., Ste 1",DUNCANVILLE,TEXAS,75116,FIRST NATIONAL BANK TEXAS,3285.0,NaN,901 E Central Texas Expy,Killeen,TX,76541,751500.0,563625.0,2022-01-05,2022,01/20/2022,Preferred Lender Program,Guaranty,4.35,120,621210.0,Offices of Dentists,NaN,NaN,DALLAS,TX,DALLAS / FT WORTH DISTRICT OFFICE,District 30,CORPORATION,New Business or 2 years or less,The status of loans that have been disburs

In [134]:
# Export SBA Loan Details to Excel
out_file_path = os.path.join(excel_exports_folder, "sba_loan_details.xlsx")
export_to_excel(sba_clean, out_file_path, sheet_name="sba_loan_details", startcol=0, startrow=0)


Exporting dataframe to excel-exports/sba_loan_details.xlsx


In [49]:
# Calc percentage of rows where _tract = '0000NAN'
sba_clean[sba_clean['_tract'] == '0000NAN'].shape[0] / sba_clean.shape[0]

0.162874251497006

In [50]:
sba_intermed_df = sba_clean\
.groupby(['county_final','_tract', 'institution_name'],
         as_index = False)\
[['Borrower name']].count().rename(\
    columns = {'Borrower name':'sba_loan_count'})

sba_intermed_df.head(10)

,county_final,_tract,institution_name,sba_loan_count
0,Collin County,0000NAN,BANK FIVE NINE,1
1,Collin County,0000NAN,BAYFIRST NATIONAL BANK,1
2,Collin County,0000NAN,BYLINE BANK,1
3,Collin County,0000NAN,CELTIC BANK CORPORATION,1
4,Collin County,0000NAN,CRAFT BANK,1
5,Collin County,0000NAN,CUSTOMERS BANK,2
6,Collin County,0000NAN,FIRST UNITED BANK AND TRUST COMPANY,2
7,Collin County,0000NAN,FROST BANK,2
8,Collin County,0000NAN,GBANK,1
9,Collin County,0000NAN,HANMI BANK,2


In [51]:
# merge hmda grouping to ffiec grouping
ffiec_hmda_intermed_df = pd.merge(
    ffiec_intermed_df,
    hmda_intermed_df,
    left_on = ['County', 'Census tract'],
    right_on = ['county_code','census_tract'],
    how = 'outer')

ffiec_hmda_intermed_df.head(10)

,index,State,County,Census tract,Tract Income Indicator,Census_Tract_No_Decimal,county_code,census_tract,institution_name,number_of_hmda_loans
0,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,21ST MORTGAGE CORPORATION,3.0
1,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,ALLIANCE BANK,1.0
2,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,AMERICAN BANK OF COMMERCE,2.0
3,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,AMERICAN FINANCIAL NETWORK INC,1.0
4,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,AMERISAVE MORTGAGE CORPORATION,1.0
5,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,ANGEL OAK HOME LOANS LLC,1.0
6,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,ASPIRE FINANCIAL INC,1.0
7,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,BANK OF ENGLAND,1.0
8,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,BETTER MORTGAGE CORPORATION,1.0
9,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,BOKF NA,1.0


In [52]:
# Summary of number_of_hmda_loans variable
print(ffiec_hmda_intermed_df['number_of_hmda_loans'].describe())

# How many nulls?
num_nulls = ffiec_hmda_intermed_df['number_of_hmda_loans'].isnull().sum()
print(f"\nNumber of nulls: {num_nulls}")

count    61253.000000
mean         1.941325
std          4.414991
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        541.000000
Name: number_of_hmda_loans, dtype: float64

Number of nulls: 37


In [53]:
# merge sba to ffiec hmda merge
ffiec_hmda_sba_intermed_df = pd.merge(ffiec_hmda_intermed_df,
                                      sba_intermed_df,
                                      left_on = ['County', 'Census tract', 'institution_name'],
                                      right_on = ['county_final','_tract', 'institution_name'],
                                      how = 'outer')

ffiec_hmda_sba_intermed_df.head(3)

,index,State,County,Census tract,Tract Income Indicator,Census_Tract_No_Decimal,county_code,census_tract,institution_name,number_of_hmda_loans,county_final,_tract,sba_loan_count
0,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,21ST MORTGAGE CORPORATION,3.0,NaN,NaN,NaN
1,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,ALLIANCE BANK,1.0,NaN,NaN,NaN
2,72753.0,TEXAS,Collin County,0301.01,Middle Income,030101,Collin County,0301.01,AMERICAN BANK OF COMMERCE,2.0,NaN,NaN,NaN


In [54]:
ffiec_hmda_sba_intermed_df['sba_loan_count'].value_counts()

sba_loan_count
1.0    659
2.0     65
3.0      6
4.0      3
5.0      2
6.0      1
Name: count, dtype: int64

In [55]:
# Export merged FFIEC, HMDA, and SBA data to Excel
excel_output_file = os.path.join(excel_exports_folder, "ffiec_hmda_sba_intermed_df.xlsx")
export_to_excel(ffiec_hmda_sba_intermed_df, excel_output_file, sheet_name="ffiec_hmda_sba_intermed_df", startcol=0, startrow=0)

Exporting dataframe to excel-exports/ffiec_hmda_sba_intermed_df.xlsx


In [56]:
# census_level_hmda_cat_summaries_long LONG FORMAT with institution name included in grouping
census_level_hmda_cat_summaries_long = {}
for category in category_lst: 
    # get value counts for hmda data
    # create a deep copy of the hmda lar data 
    hmda_dict_lar_df_cpy = hmda_dict_lar_df.copy()
    # apply functions to update institution names 
    hmda_dict_lar_df_cpy = clean_inst_names(hmda_dict_lar_df_cpy)
    # filter by originated loans and group by'county_code','census_tract' and'institution_name'
    temp_cat_counts = hmda_dict_lar_df_cpy[hmda_dict_lar_df_cpy['action_taken'] == 'Loan originated']\
    .groupby(['county_code','census_tract','institution_name'])[category].value_counts(dropna = False)\
    .reset_index().fillna(0)
    # create new columns for sum of rows 
    #temp_cat_counts['Total'] = temp_cat_counts.sum(axis = 1, numeric_only = True)
    # bring in ffiec subset
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec subset to hmda value counts df 
    temp_cat_summary = pd.merge(temp_cat_counts,tract_inc_ind, left_on = ['county_code','census_tract'],\
                                right_on = ['County','Census tract'], how = 'left')
    # add  in census tract with no decimal point 
    temp_cat_summary['Census_Tract_no_decimal'] = \
    temp_cat_summary['census_tract'].str.replace('.','')
    # add in hmda column
    temp_cat_summary['data_source'] = 'HMDA'
    # reorder column names
    temp_cat_summary = temp_cat_summary[\
    ['county_code','census_tract','Census_Tract_no_decimal','Tract Income Indicator','data_source'] + 
    temp_cat_counts.drop(['county_code','census_tract'],axis = 1).columns.to_list()]
    census_level_hmda_cat_summaries_long[category] = temp_cat_summary

In [57]:
census_level_hmda_cat_summaries_long['derived_loan_product_type'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,derived_loan_product_type,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Conventional:First Lien,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Conventional:First Lien,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Conventional:First Lien,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Conventional:First Lien,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Conventional:First Lien,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Conventional:First Lien,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Conventional:First Lien,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Conventional:First Lien,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,Conventional:First Lien,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,Conventional:Subordinate Lien,1


In [58]:
census_level_hmda_cat_summaries_long['applicant_age'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,applicant_age,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,65-74,1
1,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,55-64,1
2,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,45-54,1
3,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,35-44,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,45-54,2
5,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,45-54,1
6,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,55-64,1
7,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,55-64,1
8,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,35-44,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,55-64,1


In [59]:
census_level_hmda_cat_summaries_long['applicant_credit_score_type'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,applicant_credit_score_type,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Equifax Beacon 5.0,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Exempt,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Experian Fair Isaac,1
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Equifax Beacon 5.0,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Equifax Beacon 5.0,1
5,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Experian Fair Isaac,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Experian Fair Isaac,1
7,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,FICO Risk Score Classic 04,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Equifax Beacon 5.0,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,FICO Risk Score Classic 04,1


In [60]:
census_level_hmda_cat_summaries_long['applicant_sex'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,applicant_sex,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Male,2
1,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Female,1
2,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Male,1
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Female,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Male,1
5,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Male,1
6,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Male,1
7,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Male,1
8,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Male,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Male,1


In [61]:
census_level_hmda_cat_summaries_long['business_or_commercial_purpose'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,business_or_commercial_purpose,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Not primarily for a business or commercial pur...,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Exempt,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Not primarily for a business or commercial pur...,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Not primarily for a business or commercial pur...,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Not primarily for a business or commercial pur...,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Not primarily for a business or commercial pur...,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Not primarily for a business or commercial pur...,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Not primarily for a business or commercial pur...,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,Not primarily for a business or commercial pur...,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,Not primarily for a business or commercial pur...,1


In [62]:
census_level_hmda_cat_summaries_long['debt_to_income_ratio'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,debt_to_income_ratio,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,36%-<50%,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Exempt,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,36%-<50%,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,30%-<36%,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,36%-<50%,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,20%-<30%,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,36%-<50%,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,20%-<30%,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,36%-<50%,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,36%-<50%,1


In [63]:
census_level_hmda_cat_summaries_long['lien_status'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,lien_status,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Secured by a first lien,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Secured by a first lien,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Secured by a first lien,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Secured by a first lien,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Secured by a first lien,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Secured by a first lien,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Secured by a first lien,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Secured by a first lien,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,Secured by a first lien,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,Secured by a subordinate lien,1


In [64]:
census_level_hmda_cat_summaries_long['loan_purpose'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,loan_purpose,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Home purchase,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Home purchase,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Home purchase,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Home purchase,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Cash-out refinancing,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Refinancing,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Cash-out refinancing,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Home purchase,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,Home purchase,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,Home improvement,1


In [65]:
census_level_hmda_cat_summaries_long['loan_type'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,loan_type,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Conventional (not insured or guaranteed by FHA...,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Conventional (not insured or guaranteed by FHA...,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Conventional (not insured or guaranteed by FHA...,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Conventional (not insured or guaranteed by FHA...,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Conventional (not insured or guaranteed by FHA...,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Conventional (not insured or guaranteed by FHA...,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Conventional (not insured or guaranteed by FHA...,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Conventional (not insured or guaranteed by FHA...,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,Conventional (not insured or guaranteed by FHA...,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,Conventional (not insured or guaranteed by FHA...,1


In [66]:
census_level_hmda_cat_summaries_long['open_end_line_of_credit'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,open_end_line_of_credit,count
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,Not an open-end line of credit,3
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,Exempt,1
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,Not an open-end line of credit,2
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,Not an open-end line of credit,1
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,Not an open-end line of credit,1
5,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,Not an open-end line of credit,1
6,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,Not an open-end line of credit,1
7,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,Not an open-end line of credit,1
8,Collin County,0301.01,030101,Middle Income,HMDA,BETTER MORTGAGE CORPORATION,Not an open-end line of credit,1
9,Collin County,0301.01,030101,Middle Income,HMDA,BOKF NA,Not an open-end line of credit,1


In [67]:
# census_level_sba_cat_summaries_long LONG FORMAT grouping now include BankName->Institution name

census_level_sba_cat_summaries_long = {}
for category in ['DeliveryMethod','CongressionalDistrict','BusinessType','BusinessAge']: 
    # get value counts for sba category columns
    # create deep copy of sba data 
    sba_data_cpy = sba_data.copy()
    # apply functions to update institution names
    sba_data_cpy = clean_inst_names(sba_data_cpy)
    # group sba data by 'county_final','_tract','institution_name'
    temp_cat_counts = sba_data_cpy.groupby(['county_final','_tract','institution_name'])[category].value_counts(dropna = False)\
    .reset_index().fillna(0)
    # create new columns for sum or rows
    #temp_cat_counts['Total'] = temp_cat_counts.sum(axis = 1, numeric_only = True)
    # bring in ffiec subset
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec subset to sba value counts df 
    temp_cat_summary = pd.merge(temp_cat_counts,tract_inc_ind, left_on = ['county_final','_tract'],\
                                right_on = ['County','Census tract'], how = 'left')
    # add  in census tract with no decimal point 
    temp_cat_summary['Census_Tract_no_decimal'] = \
    temp_cat_summary['_tract'].str.replace('.','')

    # add in data source column 
    temp_cat_summary['data_source'] = 'SBA'
    
    # reorder columns names 
    temp_cat_summary = temp_cat_summary[\
    ['county_final','_tract','Census_Tract_no_decimal','Tract Income Indicator','data_source'] + 
    temp_cat_counts.drop(['county_final','_tract'], axis = 1).columns.to_list()].\
    rename(columns = {'county_final':'county_code','_tract':'census_tract'})
    census_level_sba_cat_summaries_long[category] = temp_cat_summary

In [68]:
census_level_sba_cat_summaries_long['DeliveryMethod'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,DeliveryMethod,count
0,Collin County,0000NAN,0000NAN,NaN,SBA,BANK FIVE NINE,Preferred Lender Program,1
1,Collin County,0000NAN,0000NAN,NaN,SBA,BAYFIRST NATIONAL BANK,Preferred Lender Program,1
2,Collin County,0000NAN,0000NAN,NaN,SBA,BYLINE BANK,Preferred Lender Program,1
3,Collin County,0000NAN,0000NAN,NaN,SBA,CELTIC BANK CORPORATION,Other 7(a) Loan,1
4,Collin County,0000NAN,0000NAN,NaN,SBA,CRAFT BANK,Other 7(a) Loan,1
5,Collin County,0000NAN,0000NAN,NaN,SBA,CUSTOMERS BANK,Preferred Lender Program,1
6,Collin County,0000NAN,0000NAN,NaN,SBA,CUSTOMERS BANK,SBA Express,1
7,Collin County,0000NAN,0000NAN,NaN,SBA,FIRST UNITED BANK AND TRUST COMPANY,Other 7(a) Loan,1
8,Collin County,0000NAN,0000NAN,NaN,SBA,FIRST UNITED BANK AND TRUST COMPANY,Preferred Lender Program,1
9,Collin County,0000NAN,0000NAN,NaN,SBA,FROST BANK,Preferred Lender Program,1


In [69]:
census_level_sba_cat_summaries_long['CongressionalDistrict'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,CongressionalDistrict,count
0,Collin County,0000NAN,0000NAN,NaN,SBA,BANK FIVE NINE,District 03,1
1,Collin County,0000NAN,0000NAN,NaN,SBA,BAYFIRST NATIONAL BANK,District 03,1
2,Collin County,0000NAN,0000NAN,NaN,SBA,BYLINE BANK,District 03,1
3,Collin County,0000NAN,0000NAN,NaN,SBA,CELTIC BANK CORPORATION,District 03,1
4,Collin County,0000NAN,0000NAN,NaN,SBA,CRAFT BANK,District 03,1
5,Collin County,0000NAN,0000NAN,NaN,SBA,CUSTOMERS BANK,District 03,2
6,Collin County,0000NAN,0000NAN,NaN,SBA,FIRST UNITED BANK AND TRUST COMPANY,District 03,2
7,Collin County,0000NAN,0000NAN,NaN,SBA,FROST BANK,District 03,2
8,Collin County,0000NAN,0000NAN,NaN,SBA,GBANK,District 03,1
9,Collin County,0000NAN,0000NAN,NaN,SBA,HANMI BANK,District 03,2


In [70]:
census_level_sba_cat_summaries_long['BusinessType'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,BusinessType,count
0,Collin County,0000NAN,0000NAN,NaN,SBA,BANK FIVE NINE,CORPORATION,1
1,Collin County,0000NAN,0000NAN,NaN,SBA,BAYFIRST NATIONAL BANK,CORPORATION,1
2,Collin County,0000NAN,0000NAN,NaN,SBA,BYLINE BANK,PARTNERSHIP,1
3,Collin County,0000NAN,0000NAN,NaN,SBA,CELTIC BANK CORPORATION,CORPORATION,1
4,Collin County,0000NAN,0000NAN,NaN,SBA,CRAFT BANK,CORPORATION,1
5,Collin County,0000NAN,0000NAN,NaN,SBA,CUSTOMERS BANK,CORPORATION,2
6,Collin County,0000NAN,0000NAN,NaN,SBA,FIRST UNITED BANK AND TRUST COMPANY,CORPORATION,2
7,Collin County,0000NAN,0000NAN,NaN,SBA,FROST BANK,CORPORATION,2
8,Collin County,0000NAN,0000NAN,NaN,SBA,GBANK,CORPORATION,1
9,Collin County,0000NAN,0000NAN,NaN,SBA,HANMI BANK,CORPORATION,2


In [71]:
census_level_sba_cat_summaries_long['BusinessAge'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,BusinessAge,count
0,Collin County,0000NAN,0000NAN,NaN,SBA,BANK FIVE NINE,"Startup, Loan Funds will Open Business",1
1,Collin County,0000NAN,0000NAN,NaN,SBA,BAYFIRST NATIONAL BANK,Existing or more than 2 years old,1
2,Collin County,0000NAN,0000NAN,NaN,SBA,BYLINE BANK,Change of Ownership,1
3,Collin County,0000NAN,0000NAN,NaN,SBA,CELTIC BANK CORPORATION,"Startup, Loan Funds will Open Business",1
4,Collin County,0000NAN,0000NAN,NaN,SBA,CRAFT BANK,"Startup, Loan Funds will Open Business",1
5,Collin County,0000NAN,0000NAN,NaN,SBA,CUSTOMERS BANK,"Startup, Loan Funds will Open Business",2
6,Collin County,0000NAN,0000NAN,NaN,SBA,FIRST UNITED BANK AND TRUST COMPANY,"Startup, Loan Funds will Open Business",2
7,Collin County,0000NAN,0000NAN,NaN,SBA,FROST BANK,"Startup, Loan Funds will Open Business",2
8,Collin County,0000NAN,0000NAN,NaN,SBA,GBANK,Existing or more than 2 years old,1
9,Collin County,0000NAN,0000NAN,NaN,SBA,HANMI BANK,Change of Ownership,2


In [72]:
# census_level_hmda_num_summaries

census_level_hmda_num_summaries = {}
for num_field in ['loan_amount','interest_rate','origination_charges']:
    # create deep copy of hmda data 
    temp_hmda_df = hmda_dict_lar_df.copy()
    # apply functions to update institution names 
    temp_hmda_df = clean_inst_names(temp_hmda_df)
    # replace "Exempt" with np.nan in numeric fields, group and get median for hmda groupings  
    temp_hmda_df[num_field] = temp_hmda_df[num_field].replace('Exempt',np.nan).apply(float)
    temp_num_counts = temp_hmda_df.\
    groupby(['county_code','census_tract','institution_name'])[num_field].median().reset_index()
    # bring in ffiec subset 
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec data with 5 number summary df 
    temp_num_summary = pd.merge(temp_num_counts,tract_inc_ind, left_on = ['county_code','census_tract'],\
                                 right_on = ['County','Census tract'], how = 'left')
    # add in Census_Tract_no_decimal
    temp_num_summary['Census_Tract_no_decimal'] = \
    temp_num_summary['census_tract'].str.replace('.','')

    # add in data source
    temp_num_summary['data_source'] = 'HMDA'
    
    # add df to dictionary and reorder column names 
    census_level_hmda_num_summaries[num_field] =\
    temp_num_summary[['county_code','census_tract','Census_Tract_no_decimal',\
                      'Tract Income Indicator','data_source',
                      'institution_name',
                     num_field]]\
    .fillna(0).rename(columns = {num_field:'median'})
    

In [73]:
census_level_hmda_num_summaries.keys()

dict_keys(['loan_amount', 'interest_rate', 'origination_charges'])

In [74]:
census_level_hmda_num_summaries['loan_amount'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,median
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,120000.0
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,485000.0
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,290000.0
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,445000.0
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERIHOME MORTGAGE COMPANY LLC,285000.0
5,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,315000.0
6,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,435000.0
7,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,585000.0
8,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF AMERICA NA,105000.0
9,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,435000.0


In [75]:
census_level_hmda_num_summaries['interest_rate'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,median
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,7.7000
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,0.0000
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,4.3125
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,3.3750
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERIHOME MORTGAGE COMPANY LLC,4.7500
5,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,5.0000
6,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,3.6250
7,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,3.7500
8,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF AMERICA NA,0.0000
9,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,6.0000


In [76]:
census_level_hmda_num_summaries['origination_charges'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,Tract Income Indicator,data_source,institution_name,median
0,Collin County,0301.01,030101,Middle Income,HMDA,21ST MORTGAGE CORPORATION,4500.000
1,Collin County,0301.01,030101,Middle Income,HMDA,ALLIANCE BANK,0.000
2,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN BANK OF COMMERCE,4049.280
3,Collin County,0301.01,030101,Middle Income,HMDA,AMERICAN FINANCIAL NETWORK INC,8073.120
4,Collin County,0301.01,030101,Middle Income,HMDA,AMERIHOME MORTGAGE COMPANY LLC,6436.165
5,Collin County,0301.01,030101,Middle Income,HMDA,AMERISAVE MORTGAGE CORPORATION,1868.840
6,Collin County,0301.01,030101,Middle Income,HMDA,ANGEL OAK HOME LOANS LLC,6085.000
7,Collin County,0301.01,030101,Middle Income,HMDA,ASPIRE FINANCIAL INC,21314.000
8,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF AMERICA NA,0.000
9,Collin County,0301.01,030101,Middle Income,HMDA,BANK OF ENGLAND,1859.000


In [77]:
# census_level_sba_num_summaries

census_level_sba_num_summaries = {}
for num_field in ['Total loan amount',
                  'SBAGuaranteedApproval',
                  'InitialInterestRate',
                  'TermInMonths',
                  'JobsSupported']: 
    # create deep copy of sba data 
    sba_data_cpy = sba_data.copy()
    # apply functions to update institution names
    sba_data_cpy = clean_inst_names(sba_data_cpy)
    # group sba data and get median for groupings 
    temp_num_counts = sba_data_cpy.groupby([
        'county_final',
        '_tract',
        'institution_name'])[num_field].median().reset_index()
    # bring in ffiec subset
    ffiec_data_cpy = ffiec_tract_income_lookup.copy()
    tract_inc_ind = ffiec_data_cpy[['County','Census tract','Tract Income Indicator']]
    # left merge ffiec data with 5 number summary df 
    temp_num_summary = pd.merge(temp_num_counts,tract_inc_ind, left_on = ['county_final','_tract'],\
                                 right_on = ['County','Census tract'], how = 'left')
    # add in Census_Tract_no_decimal
    temp_num_summary['Census_Tract_no_decimal'] = \
    temp_num_summary['_tract'].str.replace('.','')
    # add in data source
    temp_num_summary['data_source'] = 'SBA'
    # add df to dictionary and reorder column names 
    census_level_sba_num_summaries[num_field] =\
    temp_num_summary[['county_final','_tract','Census_Tract_no_decimal','institution_name',\
                      'Tract Income Indicator','data_source',num_field]]\
    .fillna(0).rename(columns = {'county_final':'county_code',
                                 num_field:'median',
                                 '_tract':'census_tract'})

In [78]:
census_level_sba_num_summaries.keys()

dict_keys(['Total loan amount', 'SBAGuaranteedApproval', 'InitialInterestRate', 'TermInMonths', 'JobsSupported'])

In [79]:
census_level_sba_num_summaries['Total loan amount'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,institution_name,Tract Income Indicator,data_source,median
0,Collin County,0000NAN,0000NAN,BANK FIVE NINE,0,SBA,512100.0
1,Collin County,0000NAN,0000NAN,BAYFIRST NATIONAL BANK,0,SBA,150000.0
2,Collin County,0000NAN,0000NAN,BYLINE BANK,0,SBA,5000000.0
3,Collin County,0000NAN,0000NAN,CELTIC BANK CORPORATION,0,SBA,3786300.0
4,Collin County,0000NAN,0000NAN,CRAFT BANK,0,SBA,286000.0
5,Collin County,0000NAN,0000NAN,CUSTOMERS BANK,0,SBA,565500.0
6,Collin County,0000NAN,0000NAN,FIRST UNITED BANK AND TRUST COMPANY,0,SBA,2442500.0
7,Collin County,0000NAN,0000NAN,FROST BANK,0,SBA,245000.0
8,Collin County,0000NAN,0000NAN,GBANK,0,SBA,1383300.0
9,Collin County,0000NAN,0000NAN,HANMI BANK,0,SBA,815500.0


In [80]:
census_level_sba_num_summaries['SBAGuaranteedApproval'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,institution_name,Tract Income Indicator,data_source,median
0,Collin County,0000NAN,0000NAN,BANK FIVE NINE,0,SBA,384075.0
1,Collin County,0000NAN,0000NAN,BAYFIRST NATIONAL BANK,0,SBA,127500.0
2,Collin County,0000NAN,0000NAN,BYLINE BANK,0,SBA,3750000.0
3,Collin County,0000NAN,0000NAN,CELTIC BANK CORPORATION,0,SBA,2839725.0
4,Collin County,0000NAN,0000NAN,CRAFT BANK,0,SBA,214500.0
5,Collin County,0000NAN,0000NAN,CUSTOMERS BANK,0,SBA,380375.0
6,Collin County,0000NAN,0000NAN,FIRST UNITED BANK AND TRUST COMPANY,0,SBA,1831875.0
7,Collin County,0000NAN,0000NAN,FROST BANK,0,SBA,148750.0
8,Collin County,0000NAN,0000NAN,GBANK,0,SBA,1037475.0
9,Collin County,0000NAN,0000NAN,HANMI BANK,0,SBA,611625.0


In [81]:
census_level_sba_num_summaries['InitialInterestRate'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,institution_name,Tract Income Indicator,data_source,median
0,Collin County,0000NAN,0000NAN,BANK FIVE NINE,0,SBA,8.750
1,Collin County,0000NAN,0000NAN,BAYFIRST NATIONAL BANK,0,SBA,10.000
2,Collin County,0000NAN,0000NAN,BYLINE BANK,0,SBA,6.500
3,Collin County,0000NAN,0000NAN,CELTIC BANK CORPORATION,0,SBA,5.750
4,Collin County,0000NAN,0000NAN,CRAFT BANK,0,SBA,6.000
5,Collin County,0000NAN,0000NAN,CUSTOMERS BANK,0,SBA,4.500
6,Collin County,0000NAN,0000NAN,FIRST UNITED BANK AND TRUST COMPANY,0,SBA,5.375
7,Collin County,0000NAN,0000NAN,FROST BANK,0,SBA,7.375
8,Collin County,0000NAN,0000NAN,GBANK,0,SBA,7.250
9,Collin County,0000NAN,0000NAN,HANMI BANK,0,SBA,5.500


In [82]:
census_level_sba_num_summaries['TermInMonths'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,institution_name,Tract Income Indicator,data_source,median
0,Collin County,0000NAN,0000NAN,BANK FIVE NINE,0,SBA,120.0
1,Collin County,0000NAN,0000NAN,BAYFIRST NATIONAL BANK,0,SBA,120.0
2,Collin County,0000NAN,0000NAN,BYLINE BANK,0,SBA,120.0
3,Collin County,0000NAN,0000NAN,CELTIC BANK CORPORATION,0,SBA,312.0
4,Collin County,0000NAN,0000NAN,CRAFT BANK,0,SBA,126.0
5,Collin County,0000NAN,0000NAN,CUSTOMERS BANK,0,SBA,120.0
6,Collin County,0000NAN,0000NAN,FIRST UNITED BANK AND TRUST COMPANY,0,SBA,218.0
7,Collin County,0000NAN,0000NAN,FROST BANK,0,SBA,102.0
8,Collin County,0000NAN,0000NAN,GBANK,0,SBA,299.0
9,Collin County,0000NAN,0000NAN,HANMI BANK,0,SBA,120.0


In [83]:
census_level_sba_num_summaries['JobsSupported'].head(10)

,county_code,census_tract,Census_Tract_no_decimal,institution_name,Tract Income Indicator,data_source,median
0,Collin County,0000NAN,0000NAN,BANK FIVE NINE,0,SBA,35.0
1,Collin County,0000NAN,0000NAN,BAYFIRST NATIONAL BANK,0,SBA,7.0
2,Collin County,0000NAN,0000NAN,BYLINE BANK,0,SBA,300.0
3,Collin County,0000NAN,0000NAN,CELTIC BANK CORPORATION,0,SBA,11.0
4,Collin County,0000NAN,0000NAN,CRAFT BANK,0,SBA,12.0
5,Collin County,0000NAN,0000NAN,CUSTOMERS BANK,0,SBA,40.0
6,Collin County,0000NAN,0000NAN,FIRST UNITED BANK AND TRUST COMPANY,0,SBA,59.0
7,Collin County,0000NAN,0000NAN,FROST BANK,0,SBA,7.5
8,Collin County,0000NAN,0000NAN,GBANK,0,SBA,2.0
9,Collin County,0000NAN,0000NAN,HANMI BANK,0,SBA,16.0


## tableau_stack_data_format function for **census level summaries**

In [84]:
lst_of_dcts = [census_level_hmda_cat_summaries_long, census_level_hmda_num_summaries, census_level_sba_cat_summaries_long, census_level_sba_num_summaries]

In [85]:
def tableau_stack_data_format(lst_of_summary_table_dicts):
    summaries_tableau_df = pd.DataFrame()
    for summary_dct in lst_of_summary_table_dicts:
        for summary_table_name in summary_dct.keys():
            temp_df = summary_dct[summary_table_name].copy()
            temp_df = temp_df[['county_code',
                               'census_tract',
                               'institution_name',
                               'Census_Tract_no_decimal',
                               'Tract Income Indicator', # remove this if not grouping by tract indicator
                               'data_source']]
            temp_df['variable_name'] = summary_table_name
            variable_type = ''
            if 'median' in summary_dct[summary_table_name].columns:
                variable_type = 'numeric'
            else:
                variable_type = 'categorical'
            temp_df['variable_type'] = variable_type
            if variable_type == 'numeric':
                temp_df['categorical_level'] = ''
            else:
                temp_df['categorical_level'] = summary_dct[summary_table_name][summary_table_name].copy()
            if variable_type == 'numeric':
                temp_df['value_type'] = 'median'
                temp_df['value'] = summary_dct[summary_table_name]['median'].copy()
            else:
                temp_df['value_type'] = 'count'
                temp_df['value'] = summary_dct[summary_table_name]['count'].copy()
            summaries_tableau_df = pd.concat([summaries_tableau_df,temp_df],ignore_index = True)
            print(summary_table_name,':table added',temp_df.shape[0],'rows added')
    return summaries_tableau_df

In [86]:
summaries_tableau_df_otp = tableau_stack_data_format(lst_of_dcts)

derived_loan_product_type :table added 69354 rows added
loan_type :table added 66751 rows added
loan_purpose :table added 75248 rows added
lien_status :table added 64184 rows added
open_end_line_of_credit :table added 63141 rows added
business_or_commercial_purpose :table added 63143 rows added
debt_to_income_ratio :table added 81863 rows added
applicant_credit_score_type :table added 80579 rows added
applicant_sex :table added 75845 rows added
applicant_age :table added 86916 rows added
loan_amount :table added 95314 rows added
interest_rate :table added 95314 rows added
origination_charges :table added 95314 rows added
DeliveryMethod :table added 779 rows added
CongressionalDistrict :table added 759 rows added
BusinessType :table added 738 rows added
BusinessAge :table added 765 rows added
Total loan amount :table added 736 rows added
SBAGuaranteedApproval :table added 736 rows added
InitialInterestRate :table added 736 rows added
TermInMonths :table added 736 rows added
JobsSupporte

In [87]:
summaries_tableau_df_otp.head(10)

,county_code,census_tract,institution_name,Census_Tract_no_decimal,Tract Income Indicator,data_source,variable_name,variable_type,categorical_level,value_type,value
0,Collin County,0301.01,21ST MORTGAGE CORPORATION,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,3.0
1,Collin County,0301.01,ALLIANCE BANK,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
2,Collin County,0301.01,AMERICAN BANK OF COMMERCE,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,2.0
3,Collin County,0301.01,AMERICAN FINANCIAL NETWORK INC,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
4,Collin County,0301.01,AMERISAVE MORTGAGE CORPORATION,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
5,Collin County,0301.01,ANGEL OAK HOME LOANS LLC,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
6,Collin County,0301.01,ASPIRE FINANCIAL INC,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
7,Collin County,0301.01,BANK OF ENGLAND,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
8,Collin County,0301.01,BETTER MORTGAGE CORPORATION,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
9,Collin County,0301.01,BOKF NA,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:Subordinate Lien,count,1.0


In [88]:
summaries_tableau_df_otp['variable_name'].value_counts()

variable_name
interest_rate                     95314
origination_charges               95314
loan_amount                       95314
applicant_age                     86916
debt_to_income_ratio              81863
applicant_credit_score_type       80579
applicant_sex                     75845
loan_purpose                      75248
derived_loan_product_type         69354
loan_type                         66751
lien_status                       64184
business_or_commercial_purpose    63143
open_end_line_of_credit           63141
DeliveryMethod                      779
BusinessAge                         765
CongressionalDistrict               759
BusinessType                        738
Total loan amount                   736
SBAGuaranteedApproval               736
InitialInterestRate                 736
TermInMonths                        736
JobsSupported                       736
Name: count, dtype: int64

In [89]:
summaries_tableau_df_otp['data_source'].value_counts()

data_source
HMDA    1012966
SBA        6721
Name: count, dtype: int64

# FFIEC : long format

In [90]:
# ffiec_data_summary_long_format = ffiec_data_summary.melt(
#     id_vars = [
#     'State',
#     'County',
#     'Census tract',
#     'Tract Income Indicator'],
#     value_vars = [
#         'Hispanic',
#         'White',
#         'Black',
#         'Asian',
#         'Other'], 
#     var_name = 'Race/Ethnicity',
#     value_name = 'Total Persons in Race Ethnicity Group')

In [91]:
# # adding columns to make help concate ffiec data to hmda, sba long format data
# ffiec_data_summary_long_format = ffiec_data_summary_long_format\
# .rename(columns = {'County':'county_code','Census tract':'census_tract'})
# ffiec_data_summary_long_format['Census_Tract_no_decimal'] = \
# ffiec_data_summary_long_format['census_tract'].str.replace('.','')
# ffiec_data_summary_long_format['data_source'] = 'FFIEC'
# ffiec_data_summary_long_format.head(5)

In [92]:
# add in census tract no decimal field
# ffiec_summary_tableau_df = ffiec_data_summary_long_format[[
#     'county_code',
#     'census_tract',
#     'Census_Tract_no_decimal',
#     'Tract Income Indicator',
#     'data_source']].copy()

In [93]:
# ffiec_summary_tableau_df['variable_name'] = 'Race/Ethnicity'
# ffiec_summary_tableau_df['variable_type'] = 'categorical'
# ffiec_summary_tableau_df['categorical_level'] = ffiec_data_summary_long_format['Race/Ethnicity']
# ffiec_summary_tableau_df['value_type'] = 'count'
# ffiec_summary_tableau_df['value'] = ffiec_data_summary_long_format['Total Persons in Race Ethnicity Group']
# ffiec_summary_tableau_df['institution_name'] = ''
# ffiec_summary_tableau_df = ffiec_summary_tableau_df[[\
#     'county_code',
#     'census_tract',	
#     'institution_name',
#     'Census_Tract_no_decimal',
#     'Tract Income Indicator',
#     'data_source',
#     'variable_name',
#     'variable_type',
#     'categorical_level',
#     'value_type',
#     'value']]
    
# ffiec_summary_tableau_df.head(10)

## **Concatenate summaries**

In [94]:
# concatenate census tract-level hmda and sba summaries in tableau format with census tract level ffiec summary in tableau format
# tract_hmda_sba_ffiec_summaries_tableau = pd.concat([summaries_tableau_df_otp,ffiec_summary_tableau_df], ignore_index = True)
tract_hmda_sba_ffiec_summaries_tableau = summaries_tableau_df_otp.copy()

tract_hmda_sba_ffiec_summaries_tableau.head(5)

,county_code,census_tract,institution_name,Census_Tract_no_decimal,Tract Income Indicator,data_source,variable_name,variable_type,categorical_level,value_type,value
0,Collin County,0301.01,21ST MORTGAGE CORPORATION,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,3.0
1,Collin County,0301.01,ALLIANCE BANK,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
2,Collin County,0301.01,AMERICAN BANK OF COMMERCE,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,2.0
3,Collin County,0301.01,AMERICAN FINANCIAL NETWORK INC,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0
4,Collin County,0301.01,AMERISAVE MORTGAGE CORPORATION,030101,Middle Income,HMDA,derived_loan_product_type,categorical,Conventional:First Lien,count,1.0


## export to excel workbook

In [98]:
# export hmda, sba, ffiec tableau formatted large table (9-10 minute run time)
excel_output_file = os.path.join(excel_exports_folder, "banking_dashboard_analysis_export_hmda_sba_ffiec_with_inst.xlsx")
export_to_excel(tract_hmda_sba_ffiec_summaries_tableau, excel_output_file, sheet_name="tract_hmda_sba_ffiec_summaries", startcol=0, startrow=0)

Exporting dataframe to excel-exports/banking_dashboard_analysis_export_hmda_sba_ffiec_with_inst.xlsx


In [99]:
# export cra 2022 subset data
excel_output_file = os.path.join(excel_exports_folder, "cra_2022_D11_subset.xlsx")
export_to_excel(cra_2022_D11_subset, excel_output_file, sheet_name="cra_2022_D11_subset", startcol=0, startrow=0)

Exporting dataframe to excel-exports/cra_2022_D11_subset.xlsx


In [100]:
# export hmda data in long format as multiple sheets(one for each summary)
excel_output_file = os.path.join(excel_exports_folder, "banking_dashboard_analysis_export_hmda.xlsx")

with pd.ExcelWriter(excel_output_file) as writer:
    for hmda_dct in [census_level_hmda_cat_summaries_long, census_level_hmda_num_summaries]:
        for key, value in hmda_dct.items():
            sheet_name = key
            print(f"Exporting results for {sheet_name}")  
            
            # Insert the item label into the A1 cell in the worksheet
            item_label = pd.Series(sheet_name).to_frame()
            item_label.to_excel(writer, sheet_name=sheet_name, startcol=0, startrow=0, index = False, header = False) 
            
            # Insert the dataframe
            value.to_excel(writer, sheet_name=sheet_name, startcol=3, startrow=0) 

Exporting results for derived_loan_product_type
Exporting results for loan_type
Exporting results for loan_purpose
Exporting results for lien_status
Exporting results for open_end_line_of_credit
Exporting results for business_or_commercial_purpose
Exporting results for debt_to_income_ratio
Exporting results for applicant_credit_score_type
Exporting results for applicant_sex
Exporting results for applicant_age
Exporting results for loan_amount
Exporting results for interest_rate
Exporting results for origination_charges


In [101]:
# export sba data in long format as multiple sheets(one for each summary)

excel_output_file = os.path.join(excel_exports_folder, "banking_dashboard_analysis_export_sba.xlsx")

with pd.ExcelWriter(excel_output_file) as writer:
    for sba_dict in [census_level_sba_cat_summaries_long, census_level_sba_num_summaries]:
        for key, value in sba_dict.items():
            sheet_name = key
            print(f"Exporting results for {sheet_name}")  
            
            # Insert the item label into the A1 cell in the worksheet
            item_label = pd.Series(sheet_name).to_frame()
            item_label.to_excel(writer, sheet_name=sheet_name, startcol=0, startrow=0, index = False, header = False) 
            
            # Insert the dataframe
            value.to_excel(writer, sheet_name=sheet_name, startcol=3, startrow=0) 


Exporting results for DeliveryMethod
Exporting results for CongressionalDistrict
Exporting results for BusinessType
Exporting results for BusinessAge
Exporting results for Total loan amount
Exporting results for SBAGuaranteedApproval
Exporting results for InitialInterestRate
Exporting results for TermInMonths
Exporting results for JobsSupported


# **Loan Characteristics (HMDA)**

## Categorical

In [102]:
# derived_loan_product_type
hmda_dict_lar_df[['derived_loan_product_type']].value_counts(normalize = True, dropna = False)

derived_loan_product_type    
Conventional:First Lien          0.740347
Conventional:Subordinate Lien    0.115593
FHA:First Lien                   0.100996
VA:First Lien                    0.042483
Other                            0.000582
Name: proportion, dtype: float64

In [103]:
# loan_type
hmda_dict_lar_df[['loan_type']].value_counts(normalize = True, dropna = False)

loan_type                                                                
Conventional (not insured or guaranteed by FHA, VA, RHS, or FSA)             0.855939
Federal Housing Administration insured (FHA)                                 0.101056
Veterans Affairs guaranteed (VA)                                             0.042483
USDA Rural Housing Service or Farm Service Agency guaranteed (RHS or FSA)    0.000521
Name: proportion, dtype: float64

In [104]:
# loan_purpose
hmda_dict_lar_df[['loan_purpose']].value_counts(normalize = True, dropna = False)

loan_purpose        
Home purchase           0.545263
Cash-out refinancing    0.208951
Refinancing             0.121561
Home improvement        0.060241
Other purpose           0.054930
Not applicable          0.009053
Name: proportion, dtype: float64

In [105]:
# lien_status
hmda_dict_lar_df[['lien_status']].value_counts(normalize = True, dropna = False)

lien_status                  
Secured by a first lien          0.884231
Secured by a subordinate lien    0.115769
Name: proportion, dtype: float64

In [106]:
# open_end_line_of_credit
hmda_dict_lar_df[['open_end_line_of_credit']].value_counts(normalize = True, dropna = False)

open_end_line_of_credit       
Not an open-end line of credit    0.895795
Open-end line of credit           0.097117
Exempt                            0.007088
Name: proportion, dtype: float64

In [107]:
# business_or_commercial_purpose
hmda_dict_lar_df[['business_or_commercial_purpose']].value_counts(normalize = True, dropna = False)

business_or_commercial_purpose                    
Not primarily for a business or commercial purpose    0.943128
Primarily for a business or commercial purpose        0.050165
Exempt                                                0.006707
Name: proportion, dtype: float64

## Numeric

In [108]:
# loan_amount
hmda_dict_lar_df['loan_amount'].describe()

count    2.492980e+05
mean     3.861450e+05
std      3.777388e+06
min      5.000000e+03
25%      1.650000e+05
50%      2.750000e+05
75%      4.050000e+05
max      6.939250e+08
Name: loan_amount, dtype: float64

In [109]:
# loan_amount 
hmda_dict_lar_df['loan_amount'].isnull().sum()

0

In [110]:
# interest_rate na value count
hmda_dict_lar_df['interest_rate'].replace('Exempt', np.nan).apply(float).describe()

count    157056.000000
mean          4.900759
std           1.565570
min           0.000000
25%           3.875000
50%           4.875000
75%           5.875000
max          14.450000
Name: interest_rate, dtype: float64

In [111]:
# interest_rate na value count
hmda_dict_lar_df[['interest_rate']].replace('Exempt', np.nan).isnull().sum()

interest_rate    92242
dtype: int64

In [112]:
# origination_charges
hmda_dict_lar_df['origination_charges'].replace('Exempt', np.nan).apply(float).describe()

count    125553.000000
mean       3312.992538
std        3701.652508
min           0.000000
25%        1055.000000
50%        2044.500000
75%        4680.000000
max       99480.000000
Name: origination_charges, dtype: float64

In [113]:
# origination_charges na value count
hmda_dict_lar_df[['origination_charges']].replace('Exempt', np.nan).isnull().sum()

origination_charges    123745
dtype: int64

# **Borrower characteristics (HMDA)** 

## Categorical

In [114]:
# debt_to_income_ratio
hmda_dict_lar_df['debt_to_income_ratio'].value_counts(normalize = True, dropna = False)

debt_to_income_ratio
Unknown     0.388387
36%-<50%    0.302899
30%-<36%    0.092387
20%-<30%    0.089559
50%-60%     0.048957
>60%        0.035752
<20%        0.034597
Exempt      0.007461
Name: proportion, dtype: float64

In [115]:
# applicant_credit_score_type
hmda_dict_lar_df['applicant_credit_score_type'].value_counts(normalize = True, dropna = False)

applicant_credit_score_type
Not applicable                        0.419185
Equifax Beacon 5.0                    0.190563
Experian Fair Isaac                   0.154157
FICO Risk Score Classic 04            0.149452
Other credit scoring model            0.054377
More than one credit scoring model    0.017758
Exempt                                0.007465
VantageScore 3.0                      0.003859
FICO Risk Score Classic 98            0.001825
VantageScore 2.0                      0.000718
NaN                                   0.000642
Name: proportion, dtype: float64

In [116]:
# applicant_sex
hmda_dict_lar_df['applicant_sex'].value_counts(normalize = True, dropna = False)

applicant_sex
Male                                                                                 0.523723
Female                                                                               0.271555
Not applicable                                                                       0.131457
Information not provided by applicant in mail, internet, or telephone application    0.072692
Applicant selected both male and female                                              0.000574
Name: proportion, dtype: float64

In [117]:
# applicant_age
hmda_dict_lar_df['applicant_age'].value_counts(normalize = True, dropna = False)

applicant_age
35-44             0.252288
45-54             0.196030
25-34             0.190014
55-64             0.121850
Not applicable    0.101389
65-74             0.059860
8888              0.034501
>74               0.023867
<25               0.020201
Name: proportion, dtype: float64

# **Loan characteristics (SBA)**

## Categorical

In [118]:
sba_data['DeliveryMethod'].value_counts(normalize = True, dropna = False)  

DeliveryMethod
Preferred Lender Program    0.695808
SBA Express                 0.203593
Other 7(a) Loan             0.065868
Community Advantage         0.026347
Other                       0.008383
Name: proportion, dtype: float64

In [119]:
sba_data['NaicsDescription'].value_counts(normalize = True, dropna = False)

NaicsDescription
Other                                                       0.479042
NaN                                                         0.088623
Full-Service Restaurants                                    0.052695
Limited-Service Restaurants                                 0.046707
Fitness and Recreational Sports Centers                     0.021557
Snack and Nonalcoholic Beverage Bars                        0.020359
Plumbing, Heating, and Air-Conditioning Contractors         0.020359
General Freight Trucking, Long Distance, Truckload          0.017964
Offices of Chiropractors                                    0.016766
Other Personal Care Services                                0.016766
Child Day Care Services                                     0.016766
General Automotive Repair                                   0.015569
Offices of Dentists                                         0.014371
Residential Remodelers                                      0.014371
All Other Special

In [120]:
sba_data['CongressionalDistrict'].value_counts(normalize = True, dropna = False)

CongressionalDistrict
District 03    0.269461
District 24    0.191617
District 12    0.147305
District 06    0.126946
District 30    0.119760
District 05    0.086228
District 32    0.035928
District 04    0.016766
District 33    0.005988
Name: proportion, dtype: float64

In [121]:
sba_data['BusinessType'].value_counts(normalize = True, dropna = False)

BusinessType
CORPORATION    0.968862
INDIVIDUAL     0.025150
PARTNERSHIP    0.005988
Name: proportion, dtype: float64

In [122]:
sba_data['BusinessAge'].value_counts(normalize = True, dropna = False)

BusinessAge
Existing or more than 2 years old         0.404790
Startup, Loan Funds will Open Business    0.256287
New Business or 2 years or less           0.211976
Change of Ownership                       0.124551
Unanswered                                0.002395
Name: proportion, dtype: float64

## Numeric

In [123]:
sba_data['Total loan amount'].describe()

count    8.350000e+02
mean     8.003880e+05
std      1.015572e+06
min      5.000000e+03
25%      1.512500e+05
50%      4.000000e+05
75%      1.000000e+06
max      5.000000e+06
Name: Total loan amount, dtype: float64

In [124]:
sba_data['SBAGuaranteedApproval'].describe()

count    8.350000e+02
mean     5.932510e+05
std      7.613289e+05
min      2.500000e+03
25%      1.275000e+05
50%      2.625000e+05
75%      7.500000e+05
max      3.750000e+06
Name: SBAGuaranteedApproval, dtype: float64

In [125]:
sba_data['InitialInterestRate'].describe()

count    835.000000
mean       6.919662
std        1.849849
min        0.000000
25%        5.500000
50%        6.500000
75%        8.250000
max       12.750000
Name: InitialInterestRate, dtype: float64

In [126]:
sba_data['TermInMonths'].describe()

count    835.000000
mean     156.210778
std       80.091606
min        1.000000
25%      120.000000
50%      120.000000
75%      126.000000
max      321.000000
Name: TermInMonths, dtype: float64

In [127]:
sba_data['JobsSupported'].describe()

count    835.000000
mean      14.365269
std       25.119942
min        0.000000
25%        4.000000
50%        7.000000
75%       16.000000
max      300.000000
Name: JobsSupported, dtype: float64

# **FFIEC Analysis**

In [128]:
ffiec_data.groupby(['CRA poverty criteria','Tract Income Indicator'])\
['Tract median family income as a percentage of the MSA/MD median family income. 2 decimal places, truncated.']\
.describe().reset_index()[['CRA poverty criteria',\
                           'Tract Income Indicator',\
                           'min','max']].sort_values(by = ['CRA poverty criteria',\
                           'Tract Income Indicator'], ascending = False)

,CRA poverty criteria,Tract Income Indicator,min,max
3,No,Upper Income,120.48,302.48
2,No,Unknown Income,0.00,0.00
1,No,Middle Income,80.01,119.93
0,No,Low or Moderate Income,24.88,79.96
